<a href="https://colab.research.google.com/github/dcownden/PerennialProblemsOfLifeWithABrain/blob/main/sequences/P2C1_Optimization/student/P2C1_Sequence2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a> &nbsp; <a href="https://kaggle.com/kernels/welcome?src=https://raw.githubusercontent.com/dcownden/PerennialProblemsOfLifeWithABrain/main/sequences/P2C1_Optimization/student/P2C1_Sequence2.ipynb" target="_parent"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" alt="Open in Kaggle"/></a>

The following is part of a test for an upcoming text book on computational neuroscience from an optimization and learning perspective. The book will start with evolution because ultimately, all aspects of the brain are shaped by evolution and, as we will see, evolution can also be seen as an optimization algorithm. We are sharing it now to get feedback on what works and what does not and the developments we should do.

___
# **2.1.2: Optimization in Two Dimensions: Estimating Gradients**

### Objective: Solve simple 2 dimensional optimization problems using perturbation methods, and connect these methods with physiological neural plasticity mechanisms.

In this sequence we will:

* Extend the perturbation based methods used in the previous sequence to the 2-dimesional case.

* Introduce a simple artificial nerual network model of behaviour for binary discrimination.

* Use the perturb -> measure -> step learning rule to optimize a stochastic strike-no-strike policy for a lurk and strike predator.

* Show how perturb -> measure -> update can be used to estimate gradients in 2 dimensions and how important gradients are when the updates to multiple parameters need to act in a coordinated fashion.

* Using the intuitions gained from going from 1-d to 2-d reflect on how well (or not) our current methods will scale to higher dimensional problems.


# Setup

Run the following cell to setup and install the various dependencies and helper functions for this .

In [ ]:
# @title Dependencies, Imports and Setup
# @markdown You don't need to worry about how this code works – but you do need to **run the cell**
!apt install libgraphviz-dev > /dev/null 2> /dev/null #colab
!pip install ipympl pygraphviz vibecheck datatops jupyterquiz ucimlrepo > /dev/null 2> /dev/null #google.colab

import asyncio
import requests
from requests.exceptions import RequestException
import numpy as np
import itertools
import collections
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
from matplotlib.colors import LogNorm
from matplotlib.animation import FuncAnimation
from matplotlib import gridspec
import seaborn as sns
import pandas as pd
import pygraphviz as pgv
import ipywidgets as widgets
import time
import logging
import random
import os
import copy
import torch
import warnings
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from io import BytesIO
from enum import Enum
from scipy.spatial.distance import cdist
from scipy.stats import norm
from scipy.optimize import minimize
from sklearn.linear_model import LinearRegression
from tabulate import tabulate
from IPython.display import display, clear_output, Markdown, HTML, Image
from jupyterquiz import display_quiz
from vibecheck import DatatopsContentReviewContainer
from pathlib import Path
from typing import List, Dict
from tqdm.notebook import tqdm
from ucimlrepo import fetch_ucirepo

data_set = fetch_ucirepo(id=80)
X = data_set.data.features.values
# Translate the data to have a minimum of 0
X_translated = X - X.min()
# Scale the data to have a range from 0 to 12 (which is 6 - (-6))
scaling_factor = 12 / (X.max() - X.min())
X_scaled = X_translated * scaling_factor
# Finally, shift the data to be centered between -6 and 6
X_final = X_scaled - 6

y = data_set.data.targets.values
rng = np.random.default_rng(seed=2021)
scramble_permutation = rng.permutation(X.shape[1])
Xs = X_final[:, scramble_permutation]
y1 = y % 2
y2 = np.array(y >= 5, dtype=y.dtype)
simple_index = ((y.flatten()==1) | (y.flatten()==0))
X_simple = Xs[simple_index]
y1_simple = y1[simple_index]
# if you only had one feature which would likely be best for discrimination
epsilon = 10
class_a_sep = np.mean(X_simple[y1_simple.flatten() == 1, :], axis=0) / (np.std(X_simple[y1_simple.flatten() == 1, :], axis=0) + epsilon)
class_b_sep = np.mean(X_simple[y1_simple.flatten() == 0, :], axis=0) / (np.std(X_simple[y1_simple.flatten() == 0, :], axis=0) + epsilon)
best_feature = np.argmax(class_a_sep - class_b_sep)
print(f'Best feature is {best_feature}')
X_simple_1_feature = X_simple[:, [best_feature]]

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="matplotlib")
# random seed settings and
# getting torch to use gpu if it's there


def set_seed(seed=None, seed_torch=True):
  """
  Function that controls randomness. NumPy and random modules must be imported.

  Args:
    seed : Integer
      A non-negative integer that defines the random state. Default is `None`.
    seed_torch : Boolean
      If `True` sets the random seed for pytorch tensors, so pytorch module
      must be imported. Default is `True`.

  Returns:
    Nothing.
  """
  if seed is None:
    seed = np.random.choice(2 ** 32)
  random.seed(seed)
  np.random.seed(seed)
  if seed_torch:
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

  print(f'Random seed {seed} has been set.')


def seed_worker(worker_id):
  """
  DataLoader will reseed workers following randomness in
  multi-process data loading algorithm.

  Args:
    worker_id: integer
      ID of subprocess to seed. 0 means that
      the data will be loaded in the main process
      Refer: https://pytorch.org/docs/stable/data.html#data-loading-randomness for more details

  Returns:
    Nothing
  """
  worker_seed = torch.initial_seed() % 2**32
  np.random.seed(worker_seed)
  random.seed(worker_seed)


def set_device():
  """
  Set the device. CUDA if available, CPU otherwise

  Args:
    None

  Returns:
    Nothing
  """
  device = "cuda" if torch.cuda.is_available() else "cpu"
  if device != "cuda":
    print("This notebook isn't using and doesn't need a GPU. Good.")
  else:
    print("GPU is enabled in this notebook but not needed.")
    print("If possible, in the menu under `Runtime` -> ")
    print("`Change runtime type.`  select `CPU`")

  return device


SEED = 2021
set_seed(seed=SEED)
DEVICE = set_device()


def printmd(string):
  display(Markdown(string))


# the different utility .py files used in this notebook
filenames = []
# just run the code straight out of the response, no local copies needed!
for filename in filenames:
  url = f'https://raw.githubusercontent.com/dcownden/PerennialProblemsOfLifeWithABrain/main/utils/{filename}'
  response = requests.get(url)
  # Check that we got a valid response
  if response.status_code == 200:
    code = response.content.decode()
    exec(code)
  else:
    print(f'Failed to download {url}')

# environment contingent imports
try:
  print('Running in colab')
  from google.colab import output
  output.enable_custom_widget_manager()
  from google.colab import data_table
  data_table.disable_dataframe_formatter()
  #from google.colab import output as colab_output
  #colab_output.enable_custom_widget_manager()
  IN_COLAB = True
except:
  IN_COLAB = False
  print('Not running in colab')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%matplotlib widget
plt.style.use("https://raw.githubusercontent.com/dcownden/PerennialProblemsOfLifeWithABrain/main/pplb.mplstyle")
plt.ioff() #need to use plt.show() or display explicitly
logging.getLogger('matplotlib.font_manager').setLevel(logging.ERROR)


def remove_ip_clutter(fig):
  fig.canvas.header_visible = False
  fig.canvas.toolbar_visible = False
  fig.canvas.resizable = False
  fig.canvas.footer_visible = False
  fig.canvas.draw()


def content_review(notebook_section: str):
  return DatatopsContentReviewContainer(
    "",  # No text prompt
    notebook_section,
    {
      "url": "https://pmyvdlilci.execute-api.us-east-1.amazonaws.com/klab",
      "name": "neuro_book",
      "user_key": "xuk960xj",
    },
  ).render()
feedback_prefix = "P2C1_S1"

# 2.1.2.1: Optimizing a Polynomial of Two Variables Analytically

Last sequence we reviewed the general optimization problem as anything of the form: "Find some set of parameters or values, say $x$, such that some other scalar valued quantity which depends in some way on those values, i.e. a function of $x$, call it $f(x)$, is as large (or small) as possible, subject to some constraints on the possible values of $x$." In other words, we're looking for the best setting on a dial (the value of $x$) that maximizes or minimizes our outcome (the value of $f(x)$), within certain limits (constraints on $x$).

Now we want to think about how we should do this when we have more than one dial to work with. Here's an example. Find a pair of values of $(x,y)$ on in the region $[-2, 2] \times [-2, 2]$ such that $f(x,y) = 6(x-3)^2 + 6(y-3)^2 - 3xy$ is as small as possible. This can be written more tersely with symbols as:

$$\underset{x,y\in[-2,2]}{\arg \min}\ 6(x-3)^2 + 6(y-3)^2 - 3xy$$

If you have taken (and remember!) a multivariate calculus course you might recall how this sort of problem can be solved using calculus and algebra. First we take the partial derivatives,

$$
\begin{align}
\frac{\partial f}{\partial x} &= 12 (x-3) - 3y  \\
\frac{\partial f}{\partial y} &= 12 (y-3) - 3x
\end{align}
$$

then we set that equal to zero and use our algebra skills to find the pairs of $(x,y)$ values where the derivative is zero. There is precisely one such pair at $(x,y) = (4,4)$.

We also need to check the concavity (whether the curve bends upwards or downwards) to see if this zero-derivative point is a maximum or a minimum (we are looking for a minimum in this case). In the same way that the first partial derivatives give the slope of $f(x,y)$ in the direction of the two parameters, i.e. the slope of the tangent plane, the second derivative gives the curvature.

$$
\begin{bmatrix}
\frac{\partial^2 f}{\partial x^2} & \frac{\partial^2 f}{\partial x \partial y} \\
\frac{\partial^2 f}{\partial y \partial x} & \frac{\partial^2 f}{\partial y^2} \\
\end{bmatrix} = \begin{bmatrix}
12 & -3 \\
-3 & 12 \\
\end{bmatrix}
$$

In one dimension the second derivative is just a single number, and the sign of that number tells us whether the curve opens up or down. In two dimensions though the curvature is not just a single number, nor is it even just two numbers (one for $x$ one for $y$) but rather a whole grid of numbers that captures not just the curvature along the $x$ or $y$ dimension, but also capturing the effect of the interaction between $x$ and $y$ on the function value. This grid of second partial derivatives of a function is called the Hessian, and is often denoted as $H(f)$. Whether or not a critical point (where all the first partial derivatives are equal to zero) is a maximum, a minimum or neither, is determined by an aptly named scalar quantitity called the determinant. This quantity can be computed from the Hessian of our function $H(f)$ as:

$$
\begin{align}
\det(H(f)) &= \left( \frac{\partial^2 f}{\partial x^2} \right)\left( \frac{\partial^2 f}{\partial y^2} \right) - \left( \frac{\partial^2 f}{\partial x \partial y} \right)^2 \\
&= 12 \cdot 12 - (-3)^2
\end{align}
$$

In particular the sign of the determenant, evaluated at a critical point, determines the nature of the critical point:

- If $\Delta > 0$ the point is either a local minimum or maximum.
- If $\Delta < 0$, the critical point is a saddle point, which is neither a maximum nor a minimum.
- If $\Delta = 0$, the test is inconclusive, and the point's nature cannot be determined from the determinant alone

Further, if $\Delta > 0$ then $\frac{\partial^2 f}{\partial x^2}$ and $\frac{\partial^2 f}{\partial y^2}$ will have the same sign at this critical point, and $\frac{\partial^2 f}{\partial x^2} > 0$ indicates a local minimum and $\frac{\partial^2 f}{\partial x^2} > 0$ indicates a local maximum.

For this function, the Hessian and hence the determinant are constant, so the value of the determinant at the critical point $(4,4)$, is $12^2 - 3^2 = 135$.

The determinant is positive so we know we have a maximum or a minimum and $\left. \frac{\partial^2 f}{\partial x^2} \right|_{(4,4)}=12>0$ so we know that this is a minimum (which is what we were looking for). With some reasoning we might also convince ourselves that the function is always increases with distance from this critical point, so this local minimum is also the global minimum.

That's how we do things in two dimensions the calculus way. It was a little more involved than in one dimension but still fairly tractable. In general when optimizing a function of $n$ values $x_1$, $x_2$, ... $x_n$, we will have system of $n$ equations to solve to try and find critical points (where all partial derivatives are equal to zero), and then an $n \times n$ Hessian matrix, that needs to have its determinant evaluated at each of the critical points.

In [ ]:
# @markdown **Run this cell** to plot the function we are optimizing

# Define the function
def quad_poly(x, y, a=6, b=3, c=3):
    return a * (x - b)**2 + a * (y - b)**2 - c * x * y

# Generate the grid of x and y values
x = np.linspace(0, 8, 400)
y = np.linspace(0, 8, 400)
X, Y = np.meshgrid(x, y)
Z = quad_poly(X, Y)

fig, ax = plt.subplots(figsize=(8, 6))

c = ax.contourf(X, Y, Z, cmap='viridis')
#c = ax.imshow(Z_positive, extent=[x.min(), x.max(), y.min(), y.max()], origin='lower', cmap='viridis', aspect='auto', norm=LogNorm())
ax.plot(4, 4, 'rx')  # Using 'rx' to mark the point (1,1) with a red "x"

# Add a color bar to interpret the colors
fig.colorbar(c, ax=ax)

ax.set_title('Plot of $f(x,y) = 6(x-3)^2 + 6(y-3)^2 - 3xy$')
ax.set_xlabel('x')
ax.set_ylabel('y')

plt.show()

In this countour plot dark blue indicates lower values, and brighter green/yellow colors indicate higher values. Looking at the plot confirms that our analytic approach did indeed find a minimum at $(x,y) = (4,4)$. Now that we've seen how the analytic approach extends to 2-dimensions let's take a look at how our perturbation methods, propose and reject and perturb-measure-step work in 2-dimensions.

# 2.1.2.2: Optimizing a Two Dimensional Function with Propose and Reject

Below, is an interactive implementation of the propose-and-reject method applied to finding the maximum of our 2-d polynomial. Start by clicking 'Propose' to sample a new proposed $x,y$ value. Then, decide whether to 'Accept' or 'Reject' the proposed point based on a comparison with the previous point. Your goal is to find the lowest point in of this curve. 'Show History' is turned on to start so you can see all of the previous steps in the process. Similarly the 'Show Rejected' button is turned on so you can see all the tested points that have been rejected since the last accept.

In [ ]:
# @title Stepping Through Optimization with Propose and Reject
# @markdown **Run this cell** to step through finding the minimum of a 2-d quadratic function using propose and reject.
class InteractivePolynomialProposeReject:
  def __init__(self,
               a = 6,
               b = 3,
               c = 3,
               x_bounds=(0, 8),
               y_bounds=(0, 8),
               step_size=0.2, seed=None):
    self.a = a
    self.b = b
    self.c = c
    self.x_bounds = x_bounds
    self.y_bounds = y_bounds
    self.step_size = step_size
    self.rng = np.random.default_rng(seed)
    self.current_x = 2.0
    self.current_y = 1.0
    self.current_z = self.eval_function(self.current_x, self.current_y)
    self.proposed_x = None
    self.proposed_y = None
    self.proposed_z = None
    self.x_history = []
    self.y_history = []
    self.z_history = []
    self.rejected_x = []
    self.rejected_y = []
    self.total_proposals = 0
    self.acceptance_count = 0
    self.current_proposals = 0
    self.fig, self.ax = plt.subplots(figsize=(6, 4))
    self.propose = widgets.Button(description='Propose')
    self.accept = widgets.Button(description='Accept', disabled=True)
    self.reject = widgets.Button(description='Reject', disabled=True)
    self.full_step = widgets.Button(description='Do It For Me', disabled=True)
    self.take_10_steps = widgets.Button(description='Do 10 For Me', disabled=True)
    self.reset = widgets.Button(description='Reset')
    self.show_history = widgets.Checkbox(value=True, description='Show History')
    self.show_rejected = widgets.Checkbox(value=True, description='Show Rejected')
    self.in_loop = False
    self.init_plot()
    self.narration_display = widgets.Output()
    remove_ip_clutter(self.fig)
    # Arrange widgets in a layout
    buttons_layout = widgets.VBox([
        widgets.HBox([self.propose, self.accept, self.reject]),
        widgets.HBox([self.full_step, self.take_10_steps]),
        widgets.HBox([self.reset, self.show_history, self.show_rejected])])
    buttons_and_narration = widgets.HBox([buttons_layout, self.narration_display])
    self.ui = widgets.VBox([self.fig.canvas, buttons_and_narration])
    #bind actions to handlers
    self.propose.on_click(self.on_propose_clicked)
    self.accept.on_click(self.on_accept_clicked)
    self.reject.on_click(self.on_reject_clicked)
    self.full_step.on_click(self.on_full_step_clicked)
    self.take_10_steps.on_click(self.on_take_10_steps_clicked)
    self.reset.on_click(self.on_reset_clicked)
    self.show_rejected.observe(self.on_show_rejected_toggled, 'value')
    self.show_history.observe(self.on_show_history_toggled, 'value')

  def init_plot(self):
    """Initialize the plot with placeholder data."""
    x_vals = np.linspace(self.x_bounds[0], self.x_bounds[1], 400)
    y_vals = np.linspace(self.y_bounds[0], self.y_bounds[1], 400)
    X, Y = np.meshgrid(x_vals, y_vals)
    Z = self.eval_function(X, Y)
    # Initial plot commands return line objects, keep references to them
    self.contours = self.ax.contour(X, Y, Z, cmap='viridis')
    self.point_current, = self.ax.plot([self.current_x], [self.current_y], 'bo', label='Current')
    self.point_proposed, = self.ax.plot([], [], 'go', label='Proposed')  # Empty data to start
    self.points_history, = self.ax.plot([], [], 'ks', alpha=0.5, label='History')  # Empty data to start
    self.points_rejected, = self.ax.plot([], [], 'kx', label='Rejected')  # Empty data to start
    self.ax.legend()
    alpha = 0.5 if self.show_history.value else 0
    self.points_history.set_alpha(alpha)
    alpha = 0.5 if self.show_rejected.value else 0
    self.points_rejected.set_alpha(alpha)
    self.fig.canvas.draw_idle()

  def eval_function (self, x, y, a=None, b=None, c=None):
    if a is None:
      a = self.a
    if b is None:
      b = self.b
    if c is None:
      c = self.c
    return a*(x-b)**2 + a*(y-b)**2 - c*x*y

  def on_propose_clicked(self, button):
    self.total_proposals += 1
    self.current_proposals += 1
    self.averaverage_proposals = self.acceptance_count / self.total_proposals
    perturbation_x, perturbation_y = self.rng.standard_normal(size=2) * self.step_size
    self.proposed_x = self.current_x + perturbation_x
    self.proposed_y = self.current_y + perturbation_y
    self.proposed_z = self.eval_function(self.proposed_x, self.proposed_y)
    self.point_proposed.set_data([self.proposed_x], [self.proposed_y])
    with self.narration_display:
      clear_output()
      print(f'Proposed x: {self.proposed_x:.3f}, y: {self.proposed_y:.3f}, z: {self.proposed_z:.3f}.')
      print(f' Current x: {self.current_x:.3f}, y: {self.current_y:.3f}, z: {self.current_z}')
      print("Click 'Accept' or 'Reject' to accept or reject this proposed x,y?")
      print(f'Proposals evaluated since last step: {self.current_proposals}')
      print(f'Total number of proposals evaluated: {self.total_proposals}')
    if not self.in_loop:
      self.fig.canvas.draw()
      self.accept.disabled = False
      self.reject.disabled = False
      self.propose.disabled = True
      self.full_step.disabled = True

  def on_accept_clicked(self, button):
    self.acceptance_count += 1
    self.current_proposals = 0
    with self.narration_display:
      clear_output()
      print(f'Accepted x: {self.proposed_x:.3f}, y: {self.proposed_y:.3f}, z: {self.proposed_z:.3f}.')
      print(f"Click 'Propose' or 'Do It For Me' to try a new value.")
      print(f'Proposals evaluated since last step: {self.current_proposals}')
      print(f'Total number of proposals evaluated: {self.total_proposals}')
      print(f'Average proposals per step: {self.total_proposals / self.acceptance_count:.3f}')
    self.x_history.append(self.current_x)
    self.y_history.append(self.current_y)
    self.z_history.append(self.current_z)
    self.current_x = self.proposed_x
    self.current_y = self.proposed_y
    self.current_z = self.proposed_z
    self.proposed_x = None
    self.proposed_y = None
    self.proposed_z = None
    self.rejected_x = []
    self.rejected_y = []
    self.point_current.set_data([self.current_x], [self.current_y])
    self.points_history.set_data(self.x_history, self.y_history)
    self.point_proposed.set_data([], [])
    self.points_rejected.set_data([], [])
    if not self.in_loop:
      self.fig.canvas.draw()
      self.accept.disabled = True
      self.reject.disabled = True
      self.propose.disabled = False
      self.full_step.disabled = False
      self.take_10_steps.disabled = False

  def on_reject_clicked(self, button):
    self.rejected_x.append(self.proposed_x)
    self.rejected_y.append(self.proposed_y)
    with self.narration_display:
      clear_output()
      print(f'Rejected x: {self.proposed_x:.3f}, y: {self.proposed_y:.3f}, z: {self.proposed_z:.3f}.')
      print(f' Keeping x: {self.current_x:.3f}, y: {self.current_y:.3f}, z: {self.current_z:.3f}.')
      print(f"Click 'Propose' or 'Do It For Me' to try a new value.")
      print(f'Proposals evaluated since last step: {self.current_proposals}')
      print(f'Total number of proposals evaluated: {self.total_proposals}')
    self.proposed_x = None
    self.proposed_y = None
    self.point_proposed.set_data([], [])
    self.points_rejected.set_data(self.rejected_x, self.rejected_y)
    if not self.in_loop:
      self.fig.canvas.draw()
      self.accept.disabled = True
      self.reject.disabled = True
      self.propose.disabled = False
      self.full_step.disabled = False
      self.take_10_steps.disabled = False

  def on_full_step_clicked(self, button):
    # Automatically propose, accept if better, or reject
    self.full_step.disabled = True
    self.on_propose_clicked(button)
    if self.proposed_z < self.current_z:
        self.on_accept_clicked(button)
    else:
        self.on_reject_clicked(button)
    if not self.in_loop:
      self.full_step.disabled = False

  def on_take_10_steps_clicked(self, button):
    self.take_10_steps.disabled = True
    self.propose.disabled = True
    self.accept.disabled = True
    self.reject.disabled = True
    self.full_step.disabled = True
    self.in_loop = True
    for _ in range(10):
      self.on_full_step_clicked(button)
      self.fig.canvas.draw()
      time.sleep(0.6)
    self.in_loop = False
    self.take_10_steps.disabled = False
    self.propose.disabled = False
    self.full_step.disabled = False

  def on_reset_clicked(self, button):
    self.total_proposals = 0
    self.acceptance_count = 0
    self.current_proposals = 0
    with self.narration_display:
      clear_output(wait=True)
      print(f'Reset. Clearing history')
      print(f'Generating a new curve for you to find the max of.')
    self.current_x = self.rng.uniform(self.x_bounds[0], self.y_bounds[1])
    self.current_y = self.rng.uniform(self.y_bounds[0], self.y_bounds[1])
    self.current_z = self.eval_function(self.current_x, self.current_y)
    self.proposed_x = None
    self.proposed_y = None
    self.proposed_z = None
    self.x_history = []
    self.y_history = []
    self.z_history = []
    self.accept.disabled = True
    self.reject.disabled = True
    self.propose.disabled = False
    self.point_current.set_data([self.current_x], [self.current_y])
    self.points_history.set_data(self.x_history, self.y_history)
    self.point_proposed.set_data([], [])
    self.fig.canvas.draw_idle()

  def on_show_history_toggled(self, change):
    alpha = 0.5 if self.show_history.value else 0
    self.points_history.set_alpha(alpha)
    self.fig.canvas.draw_idle()

  def on_show_rejected_toggled(self, change):
    alpha = 0.5 if self.show_rejected.value else 0
    self.points_rejected.set_alpha(alpha)
    self.fig.canvas.draw_idle()


ippr = InteractivePolynomialProposeReject()
display(ippr.fig.canvas)
clear_output()
display(ippr.ui)

Depending on the shape of the function, and the initial starting point, this can work pretty well, but as in the one dimensional case there may potentially be a lot of proposals that lead to no improvement. For this particular function, which is relatively simple, propose and reject will end up accepting roughly half of the proposals, however for more exotically shaped functions this acceptance rate can be much lower. For example, if the current point is in a narrow valley, with only a very gentle downward slope, but very steep sides, then there are relatively few directions that will lead to lower values, really only steps that are oriented with this valley. Thus, once the process is in such a location we expect there to be relatively many proposals for every step taken. And, as in the one dimensional case, because we simply either accept or reject, these rejected proposal points do nothing further to guide our search for the minimizing $x$ and $y$ values, so in a sense each rejected proposal is wasted computation. In the one dimensional case we used a pertrub-measure-step approach to overcome the computational waste of rejected proposals, so that on each function evaluation we were able to make progress towards better parameter values. Now let's see what the perturb measure step approach looks like in two dimensions.

# 2.1.2.3 Optimizing a Two Dimensional Function with Perturb-Measure-Step

In [ ]:
# @title Optimization with Perturb-Measure-Step
# @markdown **Run this cell** to step through finding the minimum of 2-d quadratic function using a perturb-measure-step approach.
class InteractivePolynomialSlopeStepper:
  def __init__(self,
               a=6.0, b=3.0, c=3.0,
               x_bounds=(2, 6),
               y_bounds=(2, 6),
               test_step_size=0.025,
               param_step_size=0.025,
               adjustable_step_size=False,
               seed=None):
    #constants
    self.a, self.b, self.c = a, b, c
    self.x_bounds, self.y_bounds = x_bounds, y_bounds
    #state
    self.current_x = 3.0
    self.current_y = 2.5
    self.current_z = self.eval_function(self.current_x, self.current_y)
    self.test_x, self.test_y, self.test_z = None, None, None
    self.proposed_x, self.proposed_y, self.proposed_z = None, None, None
    self.x_history, self.y_history, self.z_history = [], [], []
    self.total_function_evaluations = 1
    self.num_steps = 0
    #buttons
    self.test_step_slider = widgets.FloatSlider(
        value=test_step_size, min=-0.5, max=0.5, step=0.01,
        description='Test Size:', readout=True, readout_format='.2f')
    self.param_step_slider = widgets.FloatSlider(
        value=param_step_size, min=0.01, max=1.5, step=0.01,
        description='Step Size:', readout=True, readout_format='.2f')
    self.adjustable_step_size = adjustable_step_size
    self.rng = np.random.default_rng(seed)
    self.perturb_x = widgets.Button(description='Perturb x')
    self.measure_x = widgets.Button(description='Measure x', disabled=True)
    self.step_x = widgets.Button(description='Step x', disabled=True)
    self.perturb_y = widgets.Button(description='Perturb y')
    self.measure_y = widgets.Button(description='Measure y', disabled=True)
    self.step_y = widgets.Button(description='Step y', disabled=True)
    self.full_step = widgets.Button(description='Do It For Me', disabled=True)
    self.take_10_steps = widgets.Button(description='Do 10 For Me', disabled=True)
    self.reset = widgets.Button(description='Reset')
    self.show_history = widgets.Checkbox(value=True, description='Show History')
    #bind buttons to action to handlers
    self.perturb_x.on_click(self.on_perturb_clicked)
    self.measure_x.on_click(self.on_measure_clicked)
    self.step_x.on_click(self.on_step_clicked)
    self.perturb_y.on_click(self.on_perturb_clicked)
    self.measure_y.on_click(self.on_measure_clicked)
    self.step_y.on_click(self.on_step_clicked)
    self.full_step.on_click(self.on_full_step_clicked)
    self.take_10_steps.on_click(self.on_take_10_steps_clicked)
    self.reset.on_click(self.on_reset_clicked)
    self.show_history.observe(self.on_show_history_toggled, 'value')
    self.param_step_slider.observe(self.on_step_slider_changed, 'value')
    self.test_step_slider.observe(self.on_test_step_slider_changed, 'value')
    #ui setup
    self.init_plot()
    self.narration_display = widgets.Output()
    remove_ip_clutter(self.fig)
    if self.adjustable_step_size:
      buttons_layout = widgets.VBox([
        widgets.HBox([self.perturb_x, self.measure_x, self.step_x]),
        widgets.HBox([self.perturb_y, self.measure_y, self.step_y]),
        widgets.HBox([self.full_step, self.take_10_steps]),
        widgets.HBox([self.x_test_step_slider, self.x_step_slider]),
        widgets.HBox([self.y_test_step_slider, self.y_step_slider]),
        widgets.HBox([self.reset, self.show_history])])
    else:
      buttons_layout = widgets.VBox([
        widgets.HBox([self.perturb_x, self.measure_x, self.step_x]),
        widgets.HBox([self.perturb_y, self.measure_y, self.step_y]),
        widgets.HBox([self.full_step, self.take_10_steps]),
        widgets.HBox([self.reset, self.show_history])])
    buttons_and_narration = widgets.HBox([buttons_layout, self.narration_display])
    self.ui = widgets.VBox([self.fig.canvas, buttons_and_narration])

  def eval_function(self, x, y, a=None, b=None, c=None):
    if a is None:
      a = self.a
    if b is None:
      b = self.b
    if c is None:
      c = self.c
    return a * (x - b)** 2 + a * (y-b)**2 - c * y * x

  def init_plot(self):
    self.fig, axs = plt.subplots(2, 2, figsize=(10, 8))
    self.ax_c = axs[1, 0]  # Contour plot axis
    self.ax_x = axs[0, 0]  # z(x), shares contour x-axis
    self.ax_y = axs[1, 1]  # z(y), shares contour y-axis
    self.ax_legend = axs[0, 1]  # Reserved for the legend
    self.ax_x.sharex(self.ax_c)
    self.ax_y.sharey(self.ax_c)
    # Contour Plot
    x_vals = np.linspace(self.x_bounds[0], self.x_bounds[1], 400)
    y_vals = np.linspace(self.y_bounds[0], self.y_bounds[1], 400)
    X, Y = np.meshgrid(x_vals, y_vals)
    Z = self.eval_function(X, Y)
    self.contours = self.ax_c.contour(X, Y, Z, cmap='viridis')
    self.point_current, = self.ax_c.plot([self.current_x],
        [self.current_y], 'bo', label='Current')
    self.vline_current, = self.ax_c.plot([self.current_x, self.current_x],
                                         self.y_bounds, 'k--', alpha=0.3)
    self.hline_current, = self.ax_c.plot(self.x_bounds, [self.current_y, self.current_y],
                                         'k--', alpha=0.3)
    self.point_proposed, = self.ax_c.plot([], [], 'go',
                                          label='Proposed')
    self.points_history, = self.ax_c.plot([], [], 'ks', alpha=0.5,
                                        label='History', markersize=4)
    alpha = 0.5 if self.show_history.value else 0
    self.points_history.set_alpha(alpha)

    #self.ax_x.set_xlabel('x')
    self.ax_x.set_ylabel('z')
    self.ax_c.set_ylabel('y')
    self.ax_c.set_xlabel('x')
    #self.ax_y.set_ylabel('y')
    self.ax_y.set_xlabel('z')

    # Adjust visibility of shared axis tick labels
    plt.setp(self.ax_x.get_xticklabels(), visible=False)
    plt.setp(self.ax_y.get_yticklabels(), visible=False)

    # Initial empty plots for marginal plots; they will be updated dynamically
    self.curve_z_of_x, = self.ax_x.plot([], [], 'r-', label='z(x) with y fixed')
    self.curve_z_of_y, = self.ax_y.plot([], [], 'b-', label='z(y) with x fixed')
    self.x_slope_line, = self.ax_x.plot([], [], 'g-')
    self.y_slope_line, = self.ax_y.plot([], [], 'g-')
    self.x_proposed_line, = self.ax_x.plot([], [], 'g--', alpha=0.5)
    self.y_proposed_line, = self.ax_y.plot([], [], 'g--', alpha=0.5)
    self.current_xz, = self.ax_x.plot([], [], 'bo')
    self.current_yz, = self.ax_y.plot([], [], 'bo')
    self.test_x_point, = self.ax_x.plot([], [], 'rx')
    self.test_y_point, = self.ax_y.plot([], [], 'rx', label='Test')
    self.xz_proposed_point, = self.ax_x.plot([], [], 'go')
    self.yz_proposed_point, = self.ax_y.plot([], [], 'go')
    self.update_z_of_x()
    self.update_z_of_y()
    self.ax_legend.axis('off')
    handles, labels = [], []
    for ax in [self.ax_c, self.ax_x, self.ax_y]:
      for handle, label in zip(*ax.get_legend_handles_labels()):
        handles.append(handle)
        labels.append(label)
    self.ax_legend.legend(handles, labels, loc='center')
    plt.tight_layout()
    self.fig.canvas.draw_idle()

  def update_z_of_x(self, fixed_y=None):
    if fixed_y is None:
      fixed_y = self.current_y
    x_vals = np.linspace(self.x_bounds[0], self.x_bounds[1], 400)
    z_vals = self.eval_function(x_vals, np.full_like(x_vals, fixed_y))
    self.curve_z_of_x.set_data(x_vals, z_vals)
    self.current_xz.set_data([self.current_x], [self.current_z])
    min_z, max_z = z_vals.min(), z_vals.max()
    padding = (max_z - min_z) * 0.04
    self.ax_x.set_ylim(min_z - padding, max_z)

  def update_z_of_y(self, fixed_x=None):
    if fixed_x is None:
      fixed_x = self.current_x
    y_vals = np.linspace(self.y_bounds[0], self.y_bounds[1], 400)
    z_vals = self.eval_function(np.full_like(y_vals, fixed_x), y_vals)
    self.curve_z_of_y.set_data(z_vals, y_vals)
    self.current_yz.set_data([self.current_z], [self.current_y])
    min_z, max_z = z_vals.min(), z_vals.max()
    padding = (max_z - min_z) * 0.04
    self.ax_y.set_xlim(min_z - padding, max_z)

  def update_measure_x(self):
    rise = self.test_z - self.current_z
    run = self.test_x - self.current_x
    self.x_slope = rise / run
    self.xz_intercept = self.current_z - self.x_slope * self.current_x
    x_vals = np.linspace(self.x_bounds[0], self.x_bounds[1], 400)
    z_vals = self.x_slope * x_vals + self.xz_intercept
    self.x_slope_line.set_data(x_vals, z_vals)
    self.proposed_x = self.current_x - self.param_step_slider.value * self.x_slope
    self.step_xz = self.xz_intercept + self.x_slope * self.proposed_x
    self.xz_proposed_point.set_data([self.proposed_x], [self.step_xz])
    z_min, z_max = self.ax_x.get_ylim()
    self.x_proposed_line.set_data([self.proposed_x, self.proposed_x], [z_min, z_max])
    if self.proposed_y is None:
      self.point_proposed.set_data([self.proposed_x], [self.current_y])
    else:
      self.point_proposed.set_data([self.proposed_x], [self.proposed_y])
    self.fig.canvas.draw_idle()

  def update_measure_y(self):
    rise = self.test_z - self.current_z
    run = self.test_y - self.current_y
    self.y_slope = rise / run
    self.yz_intercept = self.current_z - self.y_slope * self.current_y
    y_vals = np.linspace(self.y_bounds[0], self.y_bounds[1], 400)
    z_vals = self.y_slope * y_vals + self.yz_intercept
    self.y_slope_line.set_data(z_vals, y_vals)
    self.proposed_y = self.current_y - self.param_step_slider.value * self.y_slope
    self.step_yz = self.yz_intercept + self.y_slope * self.proposed_y
    self.yz_proposed_point.set_data([self.step_yz], [self.proposed_y])
    z_min, z_max = self.ax_y.get_xlim()
    self.y_proposed_line.set_data([z_min, z_max], [self.proposed_y, self.proposed_y])
    if self.proposed_x is None:
      self.point_proposed.set_data([self.current_x], [self.proposed_y])
    else:
      self.point_proposed.set_data([self.proposed_x], [self.proposed_y])
    self.fig.canvas.draw_idle()

  def on_perturb_clicked(self, button, axis=None):
    # are we perturbing x or y use axis or infer from button description
    if axis is None:
      if button.description == 'Perturb x':
        axis = 'x'
      elif button.description == 'Perturb y':
        axis = 'y'
      else:
        raise ValueError("Invalid button description, expected 'Perturb x' or 'Perturb y'")
    if axis == 'x':
      self.test_x = self.current_x + self.test_step_slider.value
      self.test_y = self.current_y
      axis_is = 'x'
    elif axis == 'y':
      self.test_y = self.current_y + self.test_step_slider.value
      self.test_x = self.current_x
      axis_is = 'y'
    else:
      raise ValueError("Invalid axis, expected 'x' or 'y'")
    # calculate test value
    self.test_z = self.eval_function(self.test_x, self.test_y)
    self.total_function_evaluations += 1
    if axis == 'x':
      self.test_x_point.set_data([self.test_x], [self.test_z])
      self.measure_x.disabled = False
      self.perturb_x.disabled = True
      self.perturb_y.disabled = True
    elif axis == 'y':
      self.test_y_point.set_data([self.test_z], [self.test_y])
      self.measure_y.disabled = False
      self.perturb_y.disabled = True
      self.perturb_x.disabled = True
    self.fig.canvas.draw_idle()
    with self.narration_display:
      clear_output(wait=True)
      print(f'Test point: x = {self.test_x:.3f}, y = {self.test_y:.3f}, z = {self.test_z:.3f}.')
      print(f"Click 'Measure' to propose a new {axis_is} value based on this test point.")

  def on_measure_clicked(self, button, axis=None):
    if axis is None:
      if button.description == 'Measure x':
        axis = 'x'
      elif button.description == 'Measure y':
        axis = 'y'
      else:
        raise ValueError("Invalid button description, expected 'Measure x' or 'Measure y'")
    if axis == 'x':
      self.update_measure_x()
      with self.narration_display:
        clear_output(wait=True)
        print(f'Based on measurement of x slope, proposed x step from {self.current_x:.3f} to {self.proposed_x:.3f}')
        print("Click 'Step x', to make the step.")
      self.step_x.disabled = False
      self.measure_x.disabled = True
    else: # axis == 'y':
      self.update_measure_y()
      with self.narration_display:
        clear_output(wait=True)
        print(f'Based on measurement of y slope, proposed y step from {self.current_y:.3f} to {self.proposed_y:.3f}')
        print("Click 'Step y', to make the step.")
        self.step_y.disabled = False
        self.measure_y.disabled = True

  def on_step_clicked(self, button):
    self.num_steps += 1
    if self.proposed_x is None and self.proposed_y is None:
      raise ValueError("At least one of proposed x and y values must be set before stepping.")
    if self.proposed_x is None:
      self.proposed_x = self.current_x
    if self.proposed_y is None:
      self.proposed_y = self.current_y
    new_z = self.eval_function(self.proposed_x, self.proposed_y)
    self.total_function_evaluations += 1
    with self.narration_display:
      clear_output(wait=True)
      print(f'Stepped to x = {self.proposed_x:.3f}, y = {self.proposed_y:.3f}, z = {new_z:.3f}.')
      print(f"Click 'Propose' or 'Do It For Me' to try a new value.")
      print(f'Total function evaluations so far: {self.total_function_evaluations}')
      print(f'Total number of steps taken so far: {self.num_steps}')
    self.x_history.append(self.current_x)
    self.y_history.append(self.current_y)
    self.z_history.append(self.current_z)
    self.points_history.set_data(self.x_history, self.y_history)
    self.current_x = self.proposed_x
    self.current_y = self.proposed_y
    self.current_z = new_z
    self.point_current.set_data([self.current_x], [self.current_y])
    self.vline_current.set_data([self.current_x, self.current_x], self.y_bounds)
    self.hline_current.set_data(self.x_bounds, [self.current_y, self.current_y])
    self.test_x, self.test_y, self.test_z = None, None, None
    self.test_x_point.set_data([], [])
    self.test_y_point.set_data([], [])
    self.proposed_x, self.proposed_y, self.proposed_z = None, None, None
    self.x_proposed_line.set_data([], [])
    self.y_proposed_line.set_data([], [])
    self.yz_proposed_point.set_data([], [])
    self.xz_proposed_point.set_data([], [])
    self.point_proposed.set_data([], [])
    self.x_slope_line.set_data([], [])
    self.y_slope_line.set_data([], [])
    self.update_z_of_x()
    self.update_z_of_y()
    self.fig.canvas.draw_idle()
    self.step_x.disabled = True
    self.step_y.disabled = True
    self.perturb_x.disabled = False
    self.perturb_y.disabled = False
    self.full_step.disabled = False
    self.take_10_steps.disabled = False

  def on_step_slider_changed(self, change):
    self.step_x = self.current_x + self.x_step_slider.value * self.slope
    self.step_y = self.slope * self.step_x + self.y_intercept
    self.step_point.set_data([self.step_x], [self.step_y])
    y_min, y_max = self.ax.get_ylim()
    self.step_vline.set_data([self.step_x, self.step_x], [y_min, self.step_y])
    self.fig.canvas.draw_idle()

  def on_test_step_slider_changed(self, change):
    if self.test_step_slider.value == 0:
      # Set to a default non-zero value
      self.test_step_slider.value = 0.01
      with self.narration_display:
        print(f"Test step size of 0 cannot be used to calculate slope. Using 0.01 to avoid division by zero in slope calculation.")
    self.test_x = self.current_x + self.test_step_slider.value
    self.test_y = self.polynomial(self.test_x)
    self.test_point.set_data([self.test_x], [self.test_y])
    if self.step_x is not None:
      rise = self.test_y - self.current_y
      run = self.test_x - self.current_x
      self.slope = rise / run
      self.y_intercept = self.current_y - self.slope * self.current_x
      x_vals = np.linspace(self.bounds[0], self.bounds[1], 400)
      y_vals = self.slope * x_vals + self.y_intercept
      self.step_line.set_data(x_vals, y_vals)
      self.step_x = self.current_x + self.x_step_slider.value * self.slope
      self.step_y = self.slope * self.step_x + self.y_intercept
      self.step_point.set_data([self.step_x], [self.step_y])
      y_min, y_max = self.ax.get_ylim()
      self.step_vline.set_data([self.step_x, self.step_x], [y_min, self.step_y])
    self.fig.canvas.draw_idle()

  def on_full_step_clicked(self, button):
    self.on_perturb_clicked(button, axis='x')
    self.on_measure_clicked(button, axis='x')
    self.on_step_clicked(button)
    self.on_perturb_clicked(button, axis='y')
    self.on_measure_clicked(button, axis='y')
    self.on_step_clicked(button)

  def on_take_10_steps_clicked(self, button):
    for _ in range(10):
      self.on_full_step_clicked(button)

  def on_reset_clicked(self, button):
    self.proposals_evaluated = 0
    with self.narration_display:
      clear_output(wait=True)
      print(f'Reset. Clearing history')
      print(f'Randomizing initial x and y values.')
    self.current_x = self.rng.uniform(self.x_bounds[0], self.x_bounds[1])
    self.current_y = self.rng.uniform(self.y_bounds[0], self.y_bounds[1])
    self.current_z = self.eval_function(self.current_x, self.current_y)
    self.point_current.set_data([self.current_x], [self.current_y])
    self.vline_current.set_data([self.current_x, self.current_x], self.y_bounds)
    self.hline_current.set_data(self.x_bounds, [self.current_y, self.current_y])
    self.test_x, self.test_y, self.test_z = None, None, None
    self.test_x_point.set_data([], [])
    self.test_y_point.set_data([], [])
    self.proposed_x, self.proposed_y, self.proposed_z = None, None, None
    self.x_proposed_line.set_data([], [])
    self.y_proposed_line.set_data([], [])
    self.yz_proposed_point.set_data([], [])
    self.xz_proposed_point.set_data([], [])
    self.point_proposed.set_data([], [])
    self.x_slope_line.set_data([], [])
    self.y_slope_line.set_data([], [])
    self.x_history, self.y_history, self.z_history = [], [], []
    self.points_history.set_data([], [])
    self.update_z_of_x()
    self.update_z_of_y()
    self.fig.canvas.draw_idle()

  def on_show_curve_toggled(self, change):
    alpha = 1 if self.show_curve.value else 0
    self.line_polynomial.set_alpha(alpha)
    self.fig.canvas.draw_idle()

  def on_show_history_toggled(self, change):
    alpha = 0.5 if self.show_history.value else 0
    self.points_history.set_alpha(alpha)
    self.fig.canvas.draw_idle()



ipss = InteractivePolynomialSlopeStepper()
display(ipss.fig.canvas)
clear_output()
display(ipss.ui)

So for this particular function, with this particular starting point, and these step sizes, the perturb-measure-step approach is a bit more effective. Specifically, when we start at the same spot and using similar step sizes, we are able to zero in on the minimum of the curve with fewer function evaluations using perturb-measure-step than with propose-accept-reject.



**Alogorithmic Thinking Exercise**

In the 'perturb-measure-step' optimization implemented above, we perturb each parameter ($x$ or $y$), measure the effect of perturbation, and then update only the perturbed parameter. Alternatively we could have perturbed each parameter $x$, measured, then perturbed $y$ and measured, and only then simultaneously steped both parameters.

1. In the implementation above how many times is the `eval_params` function called per iteration? How many times does `eval_params` need to be called to make in improvement in both parameters

  (Answer: Two, once to measure the base and once to measure the perturbation. To do this for both parameters requires four evaluations in total)
2. If we used the alternative method described above where both the $x$ and the $y$ value are updated simultaneously, after seperate measurements how many times is the `eval_params` function called per iteration? How many function evaluations are required to make in iteration in all parameters using this method?

  (Answer: Three, once to evaluate the base params, once to evaluate the perturbation in $x$ and once to evaluate the perturbation in $y$ from the same base params. In this method all )
3. If we had $n$ parameters instead of 2, how many times would the `eval_params` function need to be called to make an iteration in all parameters in implementation above?

  (Answer: $2n$, this is less efficient is some sense since a new reference point is used to evaluate the perturbation in each dimension.)
4. If we had $n$ parameters instead of 2, how many times would the `eval_params` function need to be called to make an iteration in all parameters in the alternate implementation?

  (Answer: $n+1$, this is more efficient in some sense since all perturbations are compared to a single reference point)

Little details like this in implementation can make big differences to algorithm speed.

For now though, let's see if we can apply these methods to our simple strike-no-strike problem where a lurk and strike predator must choose between striking and not strkining based on the activation level of a single sensory neuron.


#2.1.2.4: An Artificial Neural Network Model of Striking Behaviour

In the previous sequence we introduced a visual binary classification problem in which a lurking predator must choose between striking and not striking based on sensory input. The basic problem is simple. Given a sensory input pattern, in this case the intensity of a single pixel ranging from -6 to 6, determine whether to strike or not. The 'environment' of the organism consists of a data-set. There are 1125 elements in this particular data set. Each element is an $(x,y)$ pair, where $x$ is the sensory input the organism recieves, and $y$ is the correct action the organism should take. In the version of the problem we work with the organism gains a reward point if it strikes when prey is present, loses a reward point if it strikes when prey is not present, and obtains no reward if it does not strike (whether or not prey was present). You can try out this discrimination task for yourself by running the code cell below.


In [ ]:
# @markdown **Run this cell** to try out the 'strike-no-strike' discrimination task.

class InteractiveMNISTPredator():
  def __init__(self,
               features=Xs,
               labels=y,
               feedback_type='on_strike_only', seed=123):
    # Initialize dataset, settings for image scrambling and feedback
    self.features = features
    self.labels = labels
    # features is num_data_points x 64 (reshape to 8x8 for display, each cell 0-16)
    # labels is num_data_points x 1 (values 0-9 or 0/1 depending)
    self.feedback_type = feedback_type
    self.rng = np.random.default_rng(seed)
    sample_order = self.rng.permutation(self.features.shape[0])
    self.features = self.features[sample_order]
    self.labels = self.labels[sample_order]
    # initialize game state
    self.current_index = 0
    self.score = 0
    self.best_possible_score = 0
    self.successful_strikes = 0
    self.failed_strikes = 0
    self.non_strikes = 0
    # Initialize widgets
    self.strike_button = widgets.Button(description='Strike')
    self.no_strike_button = widgets.Button(description='No Strike')
    self.score_display = widgets.Output()
    self.feedback_display = widgets.Output()

    # Initialize the figure for image display
    self.fig, self.ax = plt.subplots(figsize=(4, 4))
    remove_ip_clutter(self.fig)
    self.show_next_image()
    # Bind event handlers
    self.strike_button.on_click(self.on_strike_clicked)
    self.no_strike_button.on_click(self.on_no_strike_clicked)

    # Arrange widgets in a layout
    buttons_layout = widgets.HBox([self.strike_button, self.no_strike_button])
    board_buttons = widgets.VBox([self.fig.canvas, buttons_layout])
    self.ui = widgets.HBox([board_buttons, widgets.VBox([self.score_display,
                                                         self.feedback_display])])

  def show_next_image(self):
    # Display the next image
    image = self.features[self.current_index]
    if len(image) == 64:
        image = image.reshape(8, 8)
    elif len(image) == 1:
      scalar_value = image.flatten()[0]
      # Initialize the 8x8 array with -6 (black)
      image = np.full((8, 8), -6.0)
      # Set the second ring to 6 (white)
      image[1:-1, 1:-1] = 6
      # Set the third (inner ring) back to -6 (black)
      image[2:-2, 2:-2] = -6
      # Assuming scalar_value is already in the range -6 to 6
      #print(scalar_value)
      image[3:-3, 3:-3] = scalar_value
    else:
      raise ValueError(f'Unexpected image shape: {image.shape}')
    # Display the image
    #print(image)
    self.fig.clf()
    self.ax = self.fig.add_subplot(111)
    self.ax.set_xlim(-.5, 7.5)
    self.ax.set_ylim(-0.5, 7.5)
    self.ax.set_aspect('equal')
    self.ax.axis('off')
    self.ax.imshow(image, cmap='gray', vmin=-6, vmax=6)
    self.fig.canvas.draw_idle()  # Force redraw

  def on_strike_clicked(self, button):
    self.process_decision('Strike')

  def on_no_strike_clicked(self, button):
    self.process_decision('No Strike')

  def process_decision(self, decision):
    # freeze buttons while we process
    self.strike_button.disabled = True
    self.no_strike_button.disabled = True

    # Process the user's decision, update score, and provide feedback
    correct_action = 'Strike' if self.labels[self.current_index] == 1 else 'No Strike'
    if decision == 'Strike':
      if decision == correct_action:
        self.score += 1
        self.successful_strikes += 1
      else:
        self.score -= 1
        self.failed_strikes += 1
    elif decision == 'No Strike':
      self.non_strikes += 1
      # no strike means no gain or loss
    else:
      raise ValueError(f'Unknown decision: {decision}')

    # Show feedback and score
    if (self.feedback_type == 'both' or
      (self.feedback_type == 'on_strike_only' and decision == 'Strike')):
      # Show informative feedback
      feedback = f'Your last choice: {decision}\nCorrect last choice: {correct_action}'
    else:
      # Show uninformative feedback
      feedback = 'Feedback only available after striking.'
    with self.feedback_display:
      clear_output(wait=True)
      print(feedback)

    # Show score
    with self.score_display:
      clear_output(wait=True)
      average_score = self.score / (self.current_index+1)
      print(f'Total Score: {self.score}')
      print(f'Number of Trials: {self.current_index + 1}')
      print(f'Successful Strikes: {self.successful_strikes}')
      print(f'Failed Strikes: {self.failed_strikes}')
      print(f'Non-Strikes: {self.non_strikes}')
      print(f'Average Score Per Trial: {average_score:.2f}')

    # Prepare the next image
    self.current_index += 1
    #print(self.current_index)
    self.show_next_image()
    # Re-enable buttons
    self.strike_button.disabled = False
    self.no_strike_button.disabled = False


scramble_01_imp = InteractiveMNISTPredator(features=X_simple_1_feature,
                                           labels=y1_simple, feedback_type='both')
display(scramble_01_imp.fig.canvas)
clear_output()
display(scramble_01_imp.ui)

As we saw in the previous sequence, the simple association - strike when the center pixel is relatively light, don't strike when the center pixel is dark - works well. Previously we implemented this decision rule, i.e. policy, as a simple threshold. Now, instead we will implement a decision rule using a very simple (2-parameter) artificial neural network. (Recall that the dataset that underlies this strike-no-strike decision problem is sourced from the UCI Machine Learning Repository, Alpaydin,E. and Kaynak,C. 1998. https://doi.org/10.24432/C50P49). We used this data set in the last sequence, and spent some time going over the structure of the data. The next few code cells repeat the over-view of the data from the previous sequence, as a reminder of what the data looks like.

In [ ]:
# What is the shape and type of the data we have to work with
print(f'')
print(f'Input data type: {type(X_simple_1_feature)}')
print(f'Input data shape: {X_simple_1_feature.shape}')
print(f'Output target data type: {type(y1_simple.dtype)}')
print(f'Output target shape: {y1_simple.shape}')

This data set consist of 1125 distinct $(\text{input}, \text{output})$ pairs. The goal of the organism is to learn as quickly as possible to correctly distinguish between these two different kinds of situation.

In [ ]:
# What do the ouptus look like, what is their range
print(y1_simple[:10])
print(f'max output value: {np.max(y1_simple)}')
print(f'min output value: {np.min(y1_simple)}')

In the interactive strike-no-strik setup above we used the label '1' to correspond to situations when 'strike' is the correct action and the label '0' to correspond to situations where 'no-strike' is the correct action.

In [ ]:
# What do the inputs look like, what is there range
print(X_simple_1_feature[:10])
print(f'Max input value: {np.max(X_simple_1_feature)}')
print(f'Min input value: {np.min(X_simple_1_feature)}')

In the visualization above a high 'feature value' corresponded to lighter colors in the center pixels and a lower feature value corresponded to darker center pixels. Now that we have a sense of the type and range of the input and output pairs let's visualize the whole dataset. For a simple dataset like this we can visualize the distribution of the input values conditional on the two different "correct" outputs

In [ ]:
# For scalar data always good to look at a histogram
fig, (ax1, ax2) = plt.subplots(2, 1)
remove_ip_clutter(fig)
ax1.hist(X_simple_1_feature[y1_simple.flatten() == 1])
ax1.set_title('Feature Distribution When Strike is Correct')
ax2.hist(X_simple_1_feature[y1_simple.flatten() == 0])
ax2.set_title('Feature Distribution When No-Strike is Correct')
plt.show()

In the previous sequence we were able to dial in an optimal threshold value. You can do that again if you'd like by running the code cell below and interacting with the widget.

In [ ]:
# @title Dialing in The Optimal Threshold
# @markdown **Run this cell** to dial in the optimal threshold
class InteractiveThresholdDialer:
  def __init__(self,
               features=X_simple_1_feature,
               labels=y1_simple):
    self.features = features
    self.labels = labels
    # maybe rewards have a slider ?
    self.correct_strike_r = 1
    self.false_strike_r = -1
    self.correct_non_strike_r = 0
    self.false_non_strike_r = 0
    self.bounds = [min(features)-0.5, max(features)+0.5]
    self.threshold_slider = widgets.FloatSlider(
        value=0, min=self.bounds[0], max=self.bounds[1], step=0.1,
        description='Threshold:', readout=True, readout_format='.2f')
    self.fig, (self.hist_ax, self.r_ax) = plt.subplots(2, 1, figsize=(8, 6),
                                                       sharex=True)
    self.narration_display = widgets.Output()
    self.init_plot()
    remove_ip_clutter(self.fig)

    # Arrange widgets in a layout
    self.ui = widgets.VBox([
        widgets.HBox([self.fig.canvas,]),
        widgets.HBox([self.threshold_slider, self.narration_display])
    ])
    self.threshold_slider.observe(self.on_threshold_slider_changed, 'value')
    print("We have narration display", hasattr(self, 'narration_display'))


  def eval_threshold(self, t):
    t = np.asarray(t).reshape(1, -1)  # 1 x len(t)
    features = self.features.reshape(-1, 1)  # len(features) x 1
    labels = self.labels.reshape(-1, 1)  # len(labels) x 1
    strikes = features > t # len(features) x len(t)
    non_strikes = features <= t # len(feature) x len(t)
    should_strike = labels == 1 # len(labels) x 1
    should_not_strike = labels == 0 #len(labels) x 1

    correct_strikes = strikes & should_strike #len(labels/features) x len(t)
    false_strikes = strikes & should_not_strike #len(labels/features) x len(t)
    correct_non_strikes = non_strikes & should_not_strike #len(labels/features) x len(t)
    false_non_strikes = non_strikes & should_strike #len(labels/features) x len(t)

    r = (self.correct_strike_r * np.sum(correct_strikes, axis=0) +
         self.false_strike_r * np.sum(false_strikes, axis=0) +
         self.correct_non_strike_r * np.sum(correct_non_strikes, axis=0) +
         self.false_non_strike_r * np.sum(false_non_strikes, axis=0))
    r = r.flatten()
    if len(r) == 1:
      r = r[0]
      # for single threshold evaluations
      num_correct_strikes = np.sum(correct_strikes)
      num_false_strikes = np.sum(false_strikes)
      num_correct_non_strikes = np.sum(correct_non_strikes)
      num_false_non_strikes = np.sum(false_non_strikes)
      # Prepare a text-based confusion matrix
      confusion_matrix_data = [
          ["Should Strike", num_correct_strikes, num_false_non_strikes],
          ["Should Not Strike", num_false_strikes, num_correct_non_strikes]]
      headers = ["", "Will Strike", "Will Not Strike"]
      confusion_matrix_table = tabulate(confusion_matrix_data, headers, tablefmt="grid")
      message = f"Confusion Matrix for threshold {t[0][0]:.2f}:\n{confusion_matrix_table}\n\nTotal Reward: {r}"
      self.narration_display.clear_output(wait=True)
      with self.narration_display:
        clear_output()
        print(message)
    return r

  def on_threshold_slider_changed(self, change):
    self.plot_threshold(self.threshold_slider.value)
    self.fig.canvas.draw_idle()

  def plot_threshold(self, t):
    r = self.eval_threshold(t)
    self.threshold_vline.set_data([t, t], [0, r])
    y_min, y_max = self.hist_ax.get_ylim()
    self.hist_vline.set_data([t, t], [y_min, y_max])

  def init_plot(self):
    """Initialize the plot with placeholder data."""
    t_vals = np.linspace(self.bounds[0], self.bounds[1], 400)
    r_vals = self.eval_threshold(t_vals)
    # Initial plot commands return line objects, keep references to them
    self.threshold_vline, = self.r_ax.plot([],[] ,color='r', linestyle='--', label='Threshold')
    self.hist_vline, = self.hist_ax.plot([], [], color='r', linestyle='--', label='Threshold')
    self.reward_curve, = self.r_ax.plot(t_vals, r_vals, alpha=1.0, label='Expected Reward')
    self.hist_ax.hist(self.features[self.labels.flatten() == 1], alpha=0.5, label='Strike')
    self.hist_ax.hist(self.features[self.labels.flatten() == 0], alpha=0.5, label='No Strike')
    self.hist_ax.set_xlabel('Feature Value')
    self.hist_ax.set_ylabel('Counts')

    self.fig.suptitle('Threshold Evaluation')
    self.r_ax.set_xlabel('Threshold')
    self.r_ax.set_ylabel('Total Reward\n(Over 1125 trials)')
    self.plot_threshold(self.threshold_slider.value)
    self.hist_ax.legend()
    self.r_ax.legend()
    self.fig.canvas.draw_idle()



itd = InteractiveThresholdDialer()
display(itd.fig.canvas)
clear_output()
display(itd.ui)

That was our threshold rule. This threshold rule works well, but as we can see in the widget above, the expected reward for a given threshold is a step function the threshold (this largely stems from the fact that reward is determined by performance over a finite and discrete data set.) This flatness of the reward function with respect to changes in the threshold made it difficult for our propose and reject method as we needed big proposal jumps to step over flat regions, but then we potentially spent lots of time overshooting the peak. This flatness was even more problematic for our perturb-measure-step rule, since perturb-measure-step relied upon estimating the slope of the reward function based on a small perturbation. However, the slope is flat at most points on this curve, so this pertubation approach does not not provide any useful information to guide parameter updates, unless of course perturbations are large enough that that span the width of the steps in this step function, but having such large steps introduces a host of other issues.

As we will see here (and many times throughout the book) stochasticity can provide a kind "fuzzing" effect, a softening of the hard edges of step-wise functions like this. Note that stochasticity might be introduced both through the behaviour of the organism (e.g. it strikes with a probability determined by its sensory inputs), or in the nature of the environment, (e.g. instead of computing the reward based on a finite and discrete set of 1125 input-output pairs in our dataset, we could instead have the reward be determined a random sample from a continuous distribution of input-output pairings. In either of these cases a decision rule with the exact same parameters may potentially yeild different amounts of reward on different 'runs' of the game, i.e. reward will be a random variable. In this case we shift our focus to maximising total expected reward, i.e. the average reward the organism would recieive over many iterations of the game. Note that in an evolutionary context, expected fitness is roughly what is being "optimized" for, but note that simply maximizing the expected "score" in many of the sub games of life, e.g. eating food, not being eaten, finding mates, etc. will not be what maximizes fitness. See sequence P1C3_S3). For now we'll leave our environment as totally deterministic, and introduce stochasticity into the behaviour of the organism. This will come quite naturally out of our (very simple) artifical neural network, or 'connectionist', decision rule.

As inspiration for our decision rule we imagine a very simple organism. One that has a single sensory input neuron, in turn connected to a single output neuron by single synaptic weight. If the output neuron fires (spikes) the predator strikes, and if it does not fire the predator does not strike.

We model this creature's sensory-behaviour system as follows. Let $x$ be the raw sensory input (scalar) in a given episode. We imagine that $x$ corresponds to the activation level and firing rate of a single photosensitive neuron. This input neuron is then connected by a synapse to a single output neuron. The activation level of this output neuron is computed as
$$y = wx + b$$
Here, $b$ is the (scalar) bias, or baseline activation level of the output neuron and $w$ is the strength of the synaptic weight between the input neuron and the output neuron. Is this case there is only one output and one input so $w$ is a scalar. In cases with multiple inputs and outputs we would use $\mathbf{W}$ to denote the matrix of such synaptic weights between all inputs and outputs and $\mathbf{x}$ to denote the vector of sensory inputs. (A quick notation reminder: bold lowercase letters typically represent column vectors, bold uppercase letters typically denote matrices or higher-order tensors.) We imagine that the probabilistic spiking of this output neuron determines the strike-no-strike behaviour of the organism, specifically:
$$ \Pr \{\text{strike}\} = \sigma(y) $$
$$ \Pr \{\text{no strike}\} = 1 - \sigma(y)$$

Here $\sigma(y): \frac{1}{1+e^{-y}} = \frac{e^y}{1+e^y}$ is the standard logistic (sigmoid) function.

--Picture of this very simple neural circuit here--

**Quick Math Exercise: The Connection Between Softmax Normalization and the Standard Logistic Sigmoid.**

Consider a random variable with two possible outcomes, $A$ and $B$. The probability of each outcome is determined by applying the softmax function to their respective scores, $a$ and $b$. Specifically, the probability of outcome $A$ occurring is given by the softmax formula:
$$ \Pr \{A\} = \frac{e^a}{e^a + e^b}$$
Interestingly, $\Pr\{A\}$ can be expressed in terms of the logistic (sigmoid) function, $\sigma(x)$, where $x$ is a function of the scores $a$ and $b$. Your task is to determine this function, denoted as $f(a, b)$, so that $\Pr{A} = \sigma(f(a, b))$.

(Answer: $$\frac{e^a}{e^a + e^b} = \frac{1}{1 + e^{b-a}} = \sigma(a-b)$$
The sigmoid of the difference in the scores gives the probability. This is the sense in which softmax normalization can be viewed as the multi-variate extension of the logistic sigmoid, and softmax normalization plays a the same role in multi-class regression as the the logistic sigmoid does in binary-class or logistic regression.)

By playing with the sliders in the simualtion below, you can see how changing the parameters $w$ and $b$ affects the probability of striking across the range of possible $x$ values. See if you can dial in the optimal setting for $w$ and $b$, in the interactive exercise below. (Hint, for precise parameter dialing you can use the left and right arrows on your keyboard to change the slider values of a selected slider, you don't have to use ultra-precise mouse twitches.)

In [ ]:
# @title Dialing in The Optimal Synaptic Weight and Bias Parameters
# @markdown **Run this cell** to dial in the optimal parameters for our simple neural circuit
class InteractiveANNDialer:
  def __init__(self,
               features=X_simple_1_feature,
               labels=y1_simple):
    self.features = features
    self.labels = labels
    # maybe rewards have a slider ?
    self.correct_strike_r = 1
    self.false_strike_r = -1
    self.correct_non_strike_r = 0
    self.false_non_strike_r = 0
    self.bounds = [min(features)-0.5, max(features)+0.5]
    self.w_slider = widgets.FloatSlider(
        value=0, min=-5, max=50, step=0.1,
        description='Weight:', readout=True, readout_format='.1f')
    self.b_slider = widgets.FloatSlider(
        value=0, min=-5, max=50, step=0.1,
        description='Bias:', readout=True, readout_format='.1f')
    self.fig, self.hist_ax = plt.subplots(figsize=(8, 4))
    self.strike_prob_ax = self.hist_ax.twinx()
    self.narration_display = widgets.Output()
    self.init_plot()
    remove_ip_clutter(self.fig)

    # Arrange widgets in a layout
    self.ui = widgets.VBox([
        widgets.HBox([self.fig.canvas,]),
        widgets.HBox([widgets.VBox([self.w_slider, self.b_slider]),
                      self.narration_display])
    ])
    self.w_slider.observe(self.on_slider_changed, 'value')
    self.b_slider.observe(self.on_slider_changed, 'value')

  def sigmoid(self, x):
    return 1 / (1 + np.exp(-x))

  def compute_strike_prob(self, w, b, x=None):
    if x is None:
      x = self.features[:, None, None] # (n_bacth, 1, 1)
    W, B = np.meshgrid(w, b) # (n_w, n_b)
    y = W * x + B # (n_batch, n_w, n_b)
    p_strike = self.sigmoid(y) # (n_batch, n_w, n_b)
    return p_strike

  def eval_params_expected(self, w, b, x=None, y=None, verbose=True):
    if x is None:
      x = self.features[:, None, None] # (n_bacth, 1, 1)
    p_strike = self.compute_strike_prob(w, b, x) # (n_batch, n_w, n_b)
    p_no_strike = 1 - p_strike
    if y is None:
      y = self.labels.copy() # (n_batch,)
    should_did = np.sum(p_strike[y == 1], axis=0)
    should_but_did_not = np.sum(p_no_strike[y == 1], axis=0)
    should_not_did_not = np.sum(p_no_strike[y == 0], axis=0)
    should_not_but_did = np.sum(p_strike[y == 0], axis=0)
    expected_reward = (should_did * self.correct_strike_r +
                       should_not_but_did * self.false_strike_r +
                       should_not_did_not * self.correct_non_strike_r +
                       should_but_did_not * self.false_non_strike_r)
    if expected_reward.shape == (1,1):
      # we are dealing with a single w and b not a bunch w and b values
      expected_reward = expected_reward[0,0]
      strike_prob = p_strike[:,0,0]
      confusion_matrix_data = [
          ["Should Strike", should_did[0,0], should_but_did_not[0,0]],
          ["Should Not Strike", should_not_but_did[0,0], should_not_did_not[0,0]]]
      headers = ["", "Will Strike", "Will Not Strike"]
      confusion_matrix_table = tabulate(confusion_matrix_data, headers, tablefmt="grid")
      message = f"Confusion Matrix for weight {w:.1f} and bias {b:.1f}:\n{confusion_matrix_table}\n\nTotal Expected Reward: {expected_reward}"
      self.narration_display.clear_output(wait=True)
      with self.narration_display:
        clear_output()
        print(message)
    return expected_reward

  def on_slider_changed(self, change):
    w = self.w_slider.value
    b = self.b_slider.value
    self.plot_decision_rule(w, b)
    self.fig.canvas.draw_idle()

  def plot_decision_rule(self, w, b):
    _  = self.eval_params_expected(w, b)

    feature_vals = np.linspace(self.bounds[0], self.bounds[1], 400)
    strike_probs = self.compute_strike_prob(w, b, feature_vals)
    self.strike_prob_curve.set_data(feature_vals, strike_probs)

  def init_plot(self):
    """Initialize the plot with placeholder data."""
    # Initial plot commands return line objects, keep references to them
    self.hist_ax.hist(self.features[self.labels.flatten() == 1], alpha=0.5, label='Should Strike')
    self.hist_ax.hist(self.features[self.labels.flatten() == 0], alpha=0.5, label='Should Not Strike')

    self.strike_prob_curve, = self.strike_prob_ax.plot([], [], color='r', linestyle='--', label='Strike Prob')
    self.strike_prob_ax.set_ylim(-0.01, 1.01)
    self.hist_ax.set_xlabel('Feature Value')
    self.hist_ax.set_ylabel('Counts')
    self.strike_prob_ax.set_ylabel('Strike Probability')

    self.strike_prob_ax.spines['right'].set_visible(True)
    self.strike_prob_ax.spines['right'].set_linewidth(1.2)

    self.fig.suptitle('Parameter Evaluation')
    self.plot_decision_rule(self.w_slider.value, self.b_slider.value)

    handles1, labels1 = self.hist_ax.get_legend_handles_labels()
    handles2, labels2 = self.strike_prob_ax.get_legend_handles_labels()
    self.hist_ax.legend(handles1 + handles2, labels1 + labels2,)

    self.fig.canvas.draw_idle()



iannd = InteractiveANNDialer()
display(iannd.fig.canvas)
clear_output()
display(iannd.ui)

One way to concieve of what we are doing above, is trying to get as much of the blue (should strike) cases below the striking probability curve, while keeping as much of the pink (should not strike) cases above the striking probability curve.

**Reflect:**
1. How confident are you that you have found the best possible $w$ and $b$ given the available parameter ranges?
2. What kind of strategy did you use to find good $w$ and $b$ values?

Twiddling the parameter weights using feedback on the types of errors and the changes in total expected reward is good, but to have more confidence in the process it would be nice to have an overview of the reward 'landscape' as a function of the two parameters. That would look like this:

In [ ]:
# @markdown **Run this cell** to visualize the reward landscape as a function of the parameters $w$ and $b$.
x = X_simple_1_feature.flatten()[:, None, None].copy() # (n_bacth, 1, 1)
y = y1_simple.flatten() # (n_batch)
w = np.linspace(9, 13, 100)
b = np.linspace(32, 44, 100)

def sigmoid(x):
  return 1 / (1 + np.exp(-x))

def compute_strike_prob(w, b, x):
  W, B = np.meshgrid(w, b) # (n_w, n_b)
  output = W * x + B # (n_batch, n_w, n_b)
  p_strike = sigmoid(output) # (n_batch, n_w, n_b)
  return p_strike

def eval_params_expected(w, b, x, y):
  #correct_strike_r = 1
  #false_strike_r = -1
  #correct_non_strike_r = 0
  #false_non_strike_r = 0
  p_strike = compute_strike_prob(w, b, x) # (n_batch, n_w, n_b)
  # p_no_strike = 1 - p_strike
  should_did = np.sum(p_strike[y == 1], axis=0)
  #should_but_did_not = np.sum(p_no_strike[y == 1], axis=0)
  #should_not_did_not = np.sum(p_no_strike[y == 0], axis=0)
  should_not_but_did = np.sum(p_strike[y == 0], axis=0)
  expected_reward = should_did - should_not_but_did
  return expected_reward

expected_reward = eval_params_expected(w, b, x, y)
fig, ax = plt.subplots(figsize=(10, 8))
contours = ax.contour(w, b, expected_reward, levels=30, colors='black')
ax.clabel(contours, inline=True, fontsize=8, fmt='%1.1f')
#contour = ax.contourf(w, b, expected_reward, levels=50, cmap='viridis')
#fig.colorbar(contour, ax=ax, label='Expected Reward')  # Add a colorbar to the plot

slope = 3.38
y_lim = ax.get_ylim()
line_x = np.array(ax.get_xlim())  # Use the current x-axis limits for the line's x-values
line_y = slope * line_x  # Calculate y-values based on the slope
ax.plot(line_x, line_y, 'r--', label=f'Slope = {slope:.2f}')  # Plot the line
ax.set_ylim(y_lim)  # Set the y-axis limits to the same as before
ax.set_xlabel('Weight (w)')  # Set x-axis label
ax.set_ylabel('Bias (b)')  # Set y-axis label
ax.set_title('Reward Landscape')  # Set title
plt.show()

So this tells us that there is a long (almost flat) ridge in the reward landscape roughly along the line $\frac{b}{w} = 3.8$. In other words past a certain point it's really about the relative ratio of the $b$ and the $w$ values. This makes sense as it is the ratio of these two values that determines where the inflection point of the sigmoid is, and the relative magnitude of these paramters that determines the steepness of the transition from low to high probability as a function of the senory input signal, past a certain point there is little reward to be gained from making that transition steeper. Looking at this countour plot of the reward landscape can make us more confident that we have found, if not the best possible values for $w$ and $b$, something that is at least very close to them in terms of the total expected reward achieved. Twiddling the dials though is not a particularly scalable optimization solution, so now we are going to see how our propose and reject and our perturb-measure-step approaches work in this slightly more complex two dimensional problem. First though a quick detour looking at the details of how total expected reward, the qunatity we are using the evaluate parameter pairs $(w, b)$ is computed.

A critical enabler of being able to dial in good $w$ and $b$ parameter values was having quick feedback on the total expected reward given the parameter values. In the coding exercise below we go over a two different ways of computing this value and then compare their efficiency.

In [ ]:
################################################################################
# TODO for students: Complete the lines with ... to implement a parameter
# evaluation function
raise NotImplementedError("Exercise: parameter evaluation")
################################################################################


def np_sigmoid(x):
  return 1 / (1 + np.exp(-x))


def eval_params_slow(w, b, x, y):
  """
  evaluates parameters of simple behaviour circuit given inputs and target
  outputs, use for loops to think carefully about what we're doing
  Args:
    w: (scalar) weight between sensory and output neuron
    b: (scalar) bias of behavioural output neuron
    x: (1 np.array x batch) sensory input
       (can be single input, mini-batch of inputs or the whole batch of inputs)
    y: (1 np.array x batch) target behavioural output (can be a single target,
       mini-batch of targets, or whole batch), needs to correspond to input

  Returns:
    R_bar: the average/expected reward obtained given the parameters, over the
           (mini-)batch of inputs and targets. (mini-batch could be size 1)
  """
  batch_len = x.shape[1]
  h = np.zeros((1, batch_len))
  y_hat = np.zeros((1, batch_len))
  R_total = 0
  for ii in range(batch_len):
    h[0,ii] = ...
    y_hat[0,ii] = np_sigmoid(...)
    # y_hat is our probability of striking
    # compute the expected score
    if y[0,ii] == 1:
      # supposed to strike
      # with probability y_hat creature strikes and gets a point
      R_total += y_hat[0,ii] * 1.0
      # with probability 1-y_hat creature doesn't strike and gets zero points
      R_total += (1-y_hat[0,ii]) * 0.0
    elif y[0,ii] == 0:
      # not supposed to strike
      # with probability y_hat creature strikes and loses a point
      R_total += y_hat[0,ii] * -1.0
      # with probability 1-y_hat creature doesn't strike and gets zero points
      R_total += (1-y_hat[0,ii]) * 0.0
    else:
      raise ValueError(f'Unexpected target: {y[ii,0]}')
  R_avg = R_total / batch_len
  return R_avg


def eval_params(w, b, x, y):
  """
  evaluates parameters of simple behaviour circuit given inputs and target
  outputs, use numpy broadcasting to be fast and concise
  Args:
    w: (scalar) weight between sensory and output neuron
    b: (scalar) bias of behavioural output neuron
    x: (batch x 1 np.array) sensory input
       (can be single input, mini-batch of inputs or the whole batch of inputs)
    y: (batch x 1 np.array) target behavioural output (can be a single target,
       mini-batch of targets, or whole batch), needs to correspond to input

  Returns:
    R_bar: the average/expected reward obtained given the parameters, over the
           (mini-)batch of inputs and targets. (mini-batch could be size 1)
  """
  h = w * x + b
  y_hat = np_sigmoid(h)
  y_score = np.copy(y)
  y_score[y_score == 0] = -1
  batch_expected_reward = y_score * y_hat
  R_avg = np.mean(batch_expected_reward)
  return R_avg

[*Click for solution*](https://github.com/dcownden/PerennialProblemsOfLifeWithABrain/tree/main//sequences/P2C1_Optimization/solutions/P2C1_Sequence2_Solution_d8158759.py)



In this exercise, we actually implemented the same function twice, once using a for loop to iterate over the (mini-)batch of inputs and targets, and once using NumPy broadcasting to compute the batch reward (roughly) in parallel. Run the following code cells to see what kind of computational efficiency this buys us.

In [ ]:
X_simple_1_feature.T.shape

In [ ]:
eval_params_slow(1, 0, X_simple_1_feature.T, y1_simple.T)

In [ ]:
eval_params(1, 0, X_simple_1_feature.T, y1_simple.T)

In [ ]:
%timeit eval_params_slow(1, 0, X_simple_1_feature.T, y1_simple.T)

In [ ]:
%timeit eval_params(1, 0, X_simple_1_feature.T, y1_simple.T)

In our specific example (on the specific COLAB instance we're writing this on in 2024), the broadcasting approach in Python, using NumPy, is approximately 50 times faster than using for-loops. This is because for-loops in interpreted languages like Python incur significant overhead at each iteration, but also because potential dependencies between loop iterations limit parallelization. In contrast, broadcasting allows for vectorized operations, enabling parallel execution that can harness more of the underlying hardware's computational power. NumPy enhances this efficiency further by utilizing highly optimized, pre-compiled linear algebra libraries (primarily hardware-specific implementations of BLAS and LAPACK). The same principles apply (though are sometimes less critical) in compiled languages or with Just-In-Time (JIT) compilation frameworks like JAX. Broadcasting or tensor notation not only improves performance but also maintains code brevity and alignment with mathematical notation. As scientists and programmers, we do not need to understand every detail of how the magic of computational parallelism allows for fast linear algebra, but it is crucial to appreciate the potential impacts on computational efficiency.

With that little detour into parallel computation done, let's get into using propose and reject to optimize our behaviour generating network.

# 2.1.2.5 Propose and Reject for A Simple Neural Network

Let's step through propose and reject for this simple network, using the widget below

In [ ]:
# @title Optimize a Simple Network with Propose and Reject
# @markdown **Run this cell** to step through propose and reject optimization of our simple network.
class Interactive2ParamANNProposeReject:
  def __init__(self,
               w_bounds=(9, 13),
               b_bounds=(32, 44),
               inputs=X_simple_1_feature,
               labels=y1_simple,
               step_size=0.2, seed=None):
    self.w_bounds = w_bounds
    self.b_bounds = b_bounds
    self.strike_reward = 1
    self.strike_cost = -1
    self.non_strike_reward = 0
    self.non_strike_cost = 0
    self.inputs = inputs
    self.labels = labels
    self.step_size = step_size
    self.rng = np.random.default_rng(seed)
    self.current_w = 9.5
    self.current_b = 36.0
    self.current_r, _, _ = self.eval_function(self.current_w, self.current_b)
    self.current_r = self.current_r[0,0]
    self.proposed_w = None
    self.proposed_b = None
    self.proposed_r = None
    self.w_history = []
    self.b_history = []
    self.r_history = []
    self.rejected_w = []
    self.rejected_b = []
    self.total_proposals = 0
    self.acceptance_count = 0
    self.current_proposals = 0
    self.fig, self.ax = plt.subplots(figsize=(6, 4))
    self.propose = widgets.Button(description='Propose')
    self.accept = widgets.Button(description='Accept', disabled=True)
    self.reject = widgets.Button(description='Reject', disabled=True)
    self.full_step = widgets.Button(description='Do It For Me', disabled=True)
    self.take_10_steps = widgets.Button(description='Do 10 For Me', disabled=True)
    self.keep_going = widgets.Button(description='Do It All For Me', disabled=True)
    self.reset = widgets.Button(description='Reset')
    self.show_history = widgets.Checkbox(value=True, description='Show History')
    self.show_rejected = widgets.Checkbox(value=True, description='Show Rejected')
    self.in_loop = False
    self.init_plot()
    self.narration_display = widgets.Output()
    remove_ip_clutter(self.fig)
    # Arrange widgets in a layout
    buttons_layout = widgets.VBox([
        widgets.HBox([self.propose, self.accept, self.reject]),
        widgets.HBox([self.full_step, self.take_10_steps, self.keep_going]),
        widgets.HBox([self.reset, self.show_history, self.show_rejected])])
    buttons_and_narration = widgets.HBox([buttons_layout, self.narration_display])
    self.ui = widgets.VBox([self.fig.canvas, buttons_and_narration])
    #bind actions to handlers
    self.propose.on_click(self.on_propose_clicked)
    self.accept.on_click(self.on_accept_clicked)
    self.reject.on_click(self.on_reject_clicked)
    self.full_step.on_click(self.on_full_step_clicked)
    self.take_10_steps.on_click(self.on_take_10_steps_clicked)
    self.keep_going.on_click(self.on_keep_going_clicked)
    self.reset.on_click(self.on_reset_clicked)
    self.show_rejected.observe(self.on_show_rejected_toggled, 'value')
    self.show_history.observe(self.on_show_history_toggled, 'value')

  def init_plot(self):
    """Initialize the plot with placeholder data."""
    w_vals = np.linspace(self.w_bounds[0], self.w_bounds[1], 100)
    b_vals = np.linspace(self.b_bounds[0], self.b_bounds[1], 100)
    R, W, B = self.eval_function(w_vals, b_vals)
    # Initial plot commands return line objects, keep references to them
    self.contours = self.ax.contour(W, B, R, cmap='viridis')
    self.point_current, = self.ax.plot([self.current_w], [self.current_b], 'bo', label='Current')
    self.point_proposed, = self.ax.plot([], [], 'go', label='Proposed')  # Empty data to start
    self.points_history, = self.ax.plot([], [], 'ks', alpha=0.5, label='History')  # Empty data to start
    self.points_rejected, = self.ax.plot([], [], 'kx', label='Rejected')  # Empty data to start
    self.ax.legend()
    alpha = 0.5 if self.show_history.value else 0
    self.points_history.set_alpha(alpha)
    alpha = 0.5 if self.show_rejected.value else 0
    self.points_rejected.set_alpha(alpha)
    self.ax.set_xlabel('w')
    self.ax.set_ylabel('b')
    self.fig.canvas.draw_idle()

  def eval_function (self, w, b,
                     sc=None, sr=None, nsc=None, nsr=None):
    sc = self.strike_cost if sc is None else sc
    sr = self.strike_reward if sr is None else sr
    nsc = self.non_strike_cost if nsc is None else nsc
    nsr = self.non_strike_reward if nsr is None else nsr
    inputs = self.inputs.flatten()[:, None, None].copy() # (n_bacth, 1, 1)
    targets = self.labels.flatten() # (n_batch)
    W, B = np.meshgrid(w, b) # (n_w, n_b)
    outputs = W * inputs + B # (n_batch, n_w, n_b)
    p_strike = sigmoid(outputs) # (n_batch, n_w, n_b)
    p_no_strike = 1 - p_strike
    should_did = np.sum(p_strike[targets == 1], axis=0)
    should_but_did_not = np.sum(p_no_strike[targets == 1], axis=0)
    should_not_did_not = np.sum(p_no_strike[targets == 0], axis=0)
    should_not_but_did = np.sum(p_strike[targets == 0], axis=0)
    expected_reward = (should_did * sr + should_not_but_did * sc +
                       should_but_did_not * nsc + should_not_did_not * nsr)
    return expected_reward, W, B

  def on_propose_clicked(self, button):
    self.total_proposals += 1
    self.current_proposals += 1
    perturbation_w, perturbation_b = self.rng.standard_normal(size=2) * self.step_size
    self.proposed_w = self.current_w + perturbation_w
    self.proposed_b = self.current_b + perturbation_b
    self.proposed_r, _, _ = self.eval_function(self.proposed_w, self.proposed_b)
    self.proposed_r = self.proposed_r[0,0]
    self.point_proposed.set_data([self.proposed_w], [self.proposed_b])
    with self.narration_display:
      clear_output()
      print(f'Proposed w: {self.proposed_w:.3f}, b: {self.proposed_b:.3f}, r: {self.proposed_r:.3f}.')
      print(f' Current w: {self.current_w:.3f}, b: {self.current_b:.3f}, r: {self.current_r}')
      print("Click 'Accept' or 'Reject' to accept or reject this proposed w,b")
      print(f'Proposals evaluated since last step: {self.current_proposals}')
      print(f'Total number of proposals evaluated: {self.total_proposals}')
      if self.acceptance_count > 0:
        print(f'Average proposals per step: {self.total_proposals / self.acceptance_count:.3f}')
    if not self.in_loop:
      self.fig.canvas.draw()
      self.accept.disabled = False
      self.reject.disabled = False
      self.propose.disabled = True
      self.full_step.disabled = True
      self.take_10_steps.disabled = True
      self.keep_going.disabled = True

  def on_accept_clicked(self, button):
    self.acceptance_count += 1
    self.current_proposals = 0
    with self.narration_display:
      clear_output()
      print(f'Accepted w: {self.proposed_w:.3f}, b: {self.proposed_b:.3f}, reward: {self.proposed_r:.3f}.')
      print(f"Click 'Propose' or 'Do It For Me' to try a new value.")
      print(f'Proposals evaluated since last step: {self.current_proposals}')
      print(f'Total number of proposals evaluated: {self.total_proposals}')
      print(f'Total number of proposals accepted: {self.acceptance_count}')
      print(f'Average evaluations per step taken: {self.total_proposals / self.acceptance_count:.3f}')
    self.w_history.append(self.current_w)
    self.b_history.append(self.current_b)
    self.r_history.append(self.current_r)
    self.current_w = self.proposed_w
    self.current_b = self.proposed_b
    self.current_r = self.proposed_r
    self.proposed_w = None
    self.proposed_b = None
    self.proposed_r = None
    self.rejected_w = []
    self.rejected_b = []
    self.point_current.set_data([self.current_w], [self.current_b])
    self.points_history.set_data(self.w_history, self.b_history)
    self.point_proposed.set_data([], [])
    self.points_rejected.set_data([], [])
    if not self.in_loop:
      self.fig.canvas.draw()
      self.accept.disabled = True
      self.reject.disabled = True
      self.propose.disabled = False
      self.full_step.disabled = False
      self.take_10_steps.disabled = False
      self.keep_going.disabled = False

  def on_reject_clicked(self, button):
    self.rejected_w.append(self.proposed_w)
    self.rejected_b.append(self.proposed_b)
    with self.narration_display:
      clear_output()
      print(f'Rejected w: {self.proposed_w:.3f}, b: {self.proposed_b:.3f}, reward: {self.proposed_r:.3f}.')
      print(f' Keeping w: {self.current_w:.3f}, b: {self.current_b:.3f}, reward: {self.current_r:.3f}.')
      print(f"Click 'Propose' or 'Do It For Me' to try a new value.")
      print(f'Proposals evaluated since last step: {self.current_proposals}')
      print(f'Total number of proposals evaluated: {self.total_proposals}')
      if self.acceptance_count > 0:
        print(f'Average proposals per step: {self.total_proposals / self.acceptance_count:.3f}')
    self.proposed_w = None
    self.proposed_b = None
    self.point_proposed.set_data([], [])
    self.points_rejected.set_data(self.rejected_w, self.rejected_b)
    if not self.in_loop:
      self.fig.canvas.draw()
      self.accept.disabled = True
      self.reject.disabled = True
      self.propose.disabled = False
      self.full_step.disabled = False
      self.take_10_steps.disabled = False
      self.keep_going.disabled = False

  def on_full_step_clicked(self, button):
    # Automatically propose, accept if better, or reject
    self.full_step.disabled = True
    self.propose.disabled = True
    self.accept.disabled = True
    self.reject.disabled = True
    self.keep_going.disabled = True
    self.take_10_steps.disabled = True
    self.on_propose_clicked(button)
    if self.proposed_r > self.current_r:
        self.on_accept_clicked(button)
    else:
        self.on_reject_clicked(button)
    if not self.in_loop:
      self.full_step.disabled = False
      self.propose.disabled = False
      self.take_10_steps.disabled = False
      self.keep_going.disabled = False


  def on_take_10_steps_clicked(self, button):
    self.take_10_steps.disabled = True
    self.propose.disabled = True
    self.accept.disabled = True
    self.reject.disabled = True
    self.full_step.disabled = True
    self.keep_going.disabled = True
    self.in_loop = True
    for _ in range(10):
      self.on_full_step_clicked(button)
      self.fig.canvas.draw()
      time.sleep(0.6)
    self.in_loop = False
    self.take_10_steps.disabled = False
    self.propose.disabled = False
    self.full_step.disabled = False
    self.keep_going.disabled = False

  def on_keep_going_clicked(self, button):
    self.take_10_steps.disabled = True
    self.propose.disabled = True
    self.accept.disabled = True
    self.reject.disabled = True
    self.full_step.disabled = True
    self.keep_going.disabled = True
    self.in_loop = True
    while self.current_r < 556.9:
      self.on_full_step_clicked(button)
      self.fig.canvas.draw()
      time.sleep(0.6)
    self.in_loop = False
    self.take_10_steps.disabled = False
    self.propose.disabled = False
    self.full_step.disabled = False
    self.keep_going.disabled = False

  def on_reset_clicked(self, button):
    self.total_proposals = 0
    self.acceptance_count = 0
    self.current_proposals = 0
    with self.narration_display:
      clear_output(wait=True)
      print(f'Reset. Clearing history')
      print(f'Generating a new curve for you to find the max of.')
    self.current_w = self.rng.uniform(self.w_bounds[0], self.w_bounds[1])
    self.current_b = self.rng.uniform(self.b_bounds[0], self.b_bounds[1])
    self.current_r = self.eval_function(self.current_w, self.current_b)
    self.proposed_w = None
    self.proposed_b = None
    self.proposed_r = None
    self.w_history = []
    self.b_history = []
    self.r_history = []
    self.accept.disabled = True
    self.reject.disabled = True
    self.propose.disabled = False
    self.point_current.set_data([self.current_w], [self.current_b])
    self.points_history.set_data(self.w_history, self.b_history)
    self.point_proposed.set_data([], [])
    self.fig.canvas.draw_idle()

  def on_show_history_toggled(self, change):
    alpha = 0.5 if self.show_history.value else 0
    self.points_history.set_alpha(alpha)
    self.fig.canvas.draw_idle()

  def on_show_rejected_toggled(self, change):
    alpha = 0.5 if self.show_rejected.value else 0
    self.points_rejected.set_alpha(alpha)
    self.fig.canvas.draw_idle()


i2pnnpr = Interactive2ParamANNProposeReject()
display(i2pnnpr.fig.canvas)
clear_output()
display(i2pnnpr.ui)

So this makes good progress at first, requiring roughly two evaluations per step. But later on in the process as it comes time to ascend the flat narrow ridge in the reward-parameter landscape, many more evaluations are required before a lucky proposal is made in the, now much narrower, direction of improvement. Smaller step sizes will improve the acceptance rate, but will slow down progress per step taken because the steps are smaller! Now lets see how perturb-measure-step fares on this slightly more challenging shape (i.e. a narrow ridge as contrasted with a symmetric and nicely rounded bowl of a quadratic function.)

# 2.1.2.6 Perturb-Measure-Step for a Simple Neural Network

In [ ]:
# @title Optimize a Simple Network with Perturb-Measure-Step
# @markdown **Run this cell** to step through finding the best weight and bias parameters for a 2 parameter network governing strike-no-strike behaviour.
class InteractiveWBSlopeStepper:
  def __init__(self,
               w_bounds=(9, 13),
               b_bounds=(32, 44),
               inputs=X_simple_1_feature,
               labels=y1_simple,
               step_size=0.01, seed=None):
    # constants
    self.w_bounds = w_bounds
    self.b_bounds = b_bounds
    self.strike_reward = 1
    self.strike_cost = -1
    self.non_strike_reward = 0
    self.non_strike_cost = 0
    self.inputs = inputs
    self.labels = labels
    self.rng = np.random.default_rng(seed)
    self.step_size = step_size
    # state
    self.current_w = 9.75
    self.current_b = 36.0
    self.current_r, _, _ = self.eval_function(self.current_w, self.current_b)
    self.current_r = self.current_r[0,0]
    self.total_function_evaluations = 1
    self.num_steps = 0
    self.test_b, self.test_w, self.test_r = None, None, None
    self.proposed_b, self.proposed_w, self.proposed_r = None, None, None
    self.b_history, self.w_history, self.r_history = [], [], []
    #buttons
    self.rng = np.random.default_rng(seed)
    self.perturb_w = widgets.Button(description='Perturb w')
    self.measure_w = widgets.Button(description='Measure w', disabled=True)
    self.step_w = widgets.Button(description='Step w', disabled=True)
    self.perturb_b = widgets.Button(description='Perturb b')
    self.measure_b = widgets.Button(description='Measure b', disabled=True)
    self.step_b = widgets.Button(description='Step b', disabled=True)
    self.full_step = widgets.Button(description='Do It For Me', disabled=True)
    self.take_10_steps = widgets.Button(description='Do 10 For Me', disabled=True)
    self.reset = widgets.Button(description='Reset')
    self.show_history = widgets.Checkbox(value=True, description='Show History')
    #bind buttons to action to handlers
    self.perturb_w.on_click(self.on_perturb_clicked)
    self.measure_w.on_click(self.on_measure_clicked)
    self.step_w.on_click(self.on_step_clicked)
    self.perturb_b.on_click(self.on_perturb_clicked)
    self.measure_b.on_click(self.on_measure_clicked)
    self.step_b.on_click(self.on_step_clicked)
    self.full_step.on_click(self.on_full_step_clicked)
    self.take_10_steps.on_click(self.on_take_10_steps_clicked)
    self.reset.on_click(self.on_reset_clicked)
    self.show_history.observe(self.on_show_history_toggled, 'value')
    #ui setup
    self.narration_display = widgets.Output()
    self.init_plot()
    #remove_ip_clutter(self.fig)
    buttons_layout = widgets.VBox([
      widgets.HBox([self.perturb_w, self.measure_w, self.step_w]),
      widgets.HBox([self.perturb_b, self.measure_b, self.step_b]),
      widgets.HBox([self.full_step, self.take_10_steps]),
      widgets.HBox([self.reset, self.show_history])])
    buttons_and_narration = widgets.HBox([buttons_layout, self.narration_display])
    self.ui = widgets.VBox([self.fig.canvas, buttons_and_narration])

  def eval_function (self, w, b,
                     sc=None, sr=None, nsc=None, nsr=None):
    sc = self.strike_cost if sc is None else sc
    sr = self.strike_reward if sr is None else sr
    nsc = self.non_strike_cost if nsc is None else nsc
    nsr = self.non_strike_reward if nsr is None else nsr
    inputs = self.inputs.flatten()[:, None, None].copy() # (n_bacth, 1, 1)
    targets = self.labels.flatten() # (n_batch)
    W, B = np.meshgrid(w, b) # (n_w, n_b)
    outputs = W * inputs + B # (n_batch, n_w, n_b)
    p_strike = sigmoid(outputs) # (n_batch, n_w, n_b)
    p_no_strike = 1 - p_strike
    should_did = np.sum(p_strike[targets == 1], axis=0)
    should_but_did_not = np.sum(p_no_strike[targets == 1], axis=0)
    should_not_did_not = np.sum(p_no_strike[targets == 0], axis=0)
    should_not_but_did = np.sum(p_strike[targets == 0], axis=0)
    expected_reward = (should_did * sr + should_not_but_did * sc +
                       should_but_did_not * nsc + should_not_did_not * nsr)
    return expected_reward, W, B

  def init_plot(self):
    self.fig, axs = plt.subplots(2, 2, figsize=(10, 8))
    self.ax_c = axs[1, 0]  # Contour plot axis
    self.ax_w = axs[0, 0]  # z(x), shares contour x-axis
    self.ax_b = axs[1, 1]  # z(y), shares contour y-axis
    self.ax_legend = axs[0, 1]  # Reserved for the legend
    self.ax_w.sharex(self.ax_c)
    self.ax_b.sharey(self.ax_c)
    # Contour Plot
    w_vals = np.linspace(self.w_bounds[0], self.w_bounds[1], 200)
    b_vals = np.linspace(self.b_bounds[0], self.b_bounds[1], 200)
    R, W, B = self.eval_function(w_vals, b_vals)
    self.contours = self.ax_c.contour(W, B, R, cmap='viridis')
    self.point_current, = self.ax_c.plot([self.current_w],
        [self.current_b], 'bo', label='Current')
    self.vline_current, = self.ax_c.plot([self.current_w, self.current_w],
                                         self.b_bounds, 'k--', alpha=0.3)
    self.hline_current, = self.ax_c.plot(self.w_bounds, [self.current_b, self.current_b],
                                         'k--', alpha=0.3)
    self.point_proposed, = self.ax_c.plot([], [], 'go',
                                          label='Proposed')
    self.points_history, = self.ax_c.plot([], [], 'ks', alpha=0.5,
                                        label='History', markersize=4)
    alpha = 0.5 if self.show_history.value else 0
    self.points_history.set_alpha(alpha)

    #self.ax_x.set_xlabel('x')
    self.ax_w.set_ylabel('Reward')
    self.ax_c.set_ylabel('b')
    self.ax_c.set_xlabel('w')
    #self.ax_y.set_ylabel('y')
    self.ax_b.set_xlabel('Reward')

    # Adjust visibility of shared axis tick labels
    plt.setp(self.ax_w.get_xticklabels(), visible=False)
    plt.setp(self.ax_b.get_yticklabels(), visible=False)

    # Initial empty plots for marginal plots; they will be updated dynamically
    self.curve_r_of_w, = self.ax_w.plot([], [], 'r-', label='R(w) with b fixed')
    self.curve_r_of_b, = self.ax_b.plot([], [], 'b-', label='R(b) with w fixed')
    self.w_slope_line, = self.ax_w.plot([], [], 'g-')
    self.b_slope_line, = self.ax_b.plot([], [], 'g-')
    self.w_proposed_line, = self.ax_w.plot([], [], 'g--', alpha=0.5)
    self.b_proposed_line, = self.ax_b.plot([], [], 'g--', alpha=0.5)
    self.current_wr, = self.ax_w.plot([], [], 'bo')
    self.current_br, = self.ax_b.plot([], [], 'bo')
    self.test_w_point, = self.ax_w.plot([], [], 'rx')
    self.test_b_point, = self.ax_b.plot([], [], 'rx', label='Test')
    self.wr_proposed_point, = self.ax_w.plot([], [], 'go')
    self.br_proposed_point, = self.ax_b.plot([], [], 'go')
    self.update_r_of_w()
    self.update_r_of_b()
    self.ax_legend.axis('off')
    handles, labels = [], []
    for ax in [self.ax_c, self.ax_w, self.ax_b]:
      for handle, label in zip(*ax.get_legend_handles_labels()):
        handles.append(handle)
        labels.append(label)
    self.ax_legend.legend(handles, labels, loc='center')
    plt.tight_layout()
    self.fig.canvas.draw_idle()

  def update_r_of_w(self, fixed_b=None):
    if fixed_b is None:
      fixed_b = self.current_b
    w_vals = np.linspace(self.w_bounds[0], self.w_bounds[1], 400)
    r_vals, _, _ = self.eval_function(w_vals, fixed_b)
    r_vals = r_vals.flatten()
    self.curve_r_of_w.set_data(w_vals, r_vals)
    self.current_wr.set_data([self.current_w], [self.current_r])
    min_r, max_r = r_vals.min(), r_vals.max()
    padding = (max_r - min_r) * 0.04
    self.ax_w.set_ylim(min_r, max_r + padding)

  def update_r_of_b(self, fixed_w=None):
    if fixed_w is None:
      fixed_w = self.current_w
    b_vals = np.linspace(self.b_bounds[0], self.b_bounds[1], 400)
    r_vals, _, _ = self.eval_function(fixed_w, b_vals)
    r_vals = r_vals.flatten()
    self.curve_r_of_b.set_data(r_vals, b_vals)
    self.current_br.set_data([self.current_r], [self.current_b])
    min_r, max_r = r_vals.min(), r_vals.max()
    padding = (max_r - min_r) * 0.04
    self.ax_b.set_xlim(min_r, max_r + padding)

  def update_measure_w(self):
    rise = self.test_r - self.current_r
    run = self.test_w - self.current_w
    self.w_slope = rise / run
    self.wr_intercept = self.current_r - self.w_slope * self.current_w
    w_vals = np.linspace(self.w_bounds[0], self.w_bounds[1], 400)
    r_vals = self.w_slope * w_vals + self.wr_intercept
    self.w_slope_line.set_data(w_vals, r_vals)
    self.proposed_w = self.current_w + self.step_size * self.w_slope
    self.step_wr = self.wr_intercept + self.w_slope * self.proposed_w
    self.wr_proposed_point.set_data([self.proposed_w], [self.step_wr])
    r_min, r_max = self.ax_w.get_ylim()
    self.w_proposed_line.set_data([self.proposed_w, self.proposed_w], [r_min, r_max])
    if self.proposed_b is None:
      self.point_proposed.set_data([self.proposed_w], [self.current_b])
    else:
      self.point_proposed.set_data([self.proposed_w], [self.proposed_b])
    self.fig.canvas.draw_idle()

  def update_measure_b(self):
    rise = self.test_r - self.current_r
    run = self.test_b - self.current_b
    self.b_slope = rise / run
    self.br_intercept = self.current_r - self.b_slope * self.current_b
    b_vals = np.linspace(self.b_bounds[0], self.b_bounds[1], 400)
    r_vals = self.b_slope * b_vals + self.br_intercept
    self.b_slope_line.set_data(r_vals, b_vals)
    self.proposed_b = self.current_b + self.step_size * self.b_slope
    self.step_br = self.br_intercept + self.b_slope * self.proposed_b
    self.br_proposed_point.set_data([self.step_br], [self.proposed_b])
    r_min, r_max = self.ax_b.get_xlim()
    self.b_proposed_line.set_data([r_min, r_max], [self.proposed_b, self.proposed_b])
    if self.proposed_w is None:
      self.point_proposed.set_data([self.current_w], [self.proposed_b])
    else:
      self.point_proposed.set_data([self.proposed_w], [self.proposed_b])
    self.fig.canvas.draw_idle()

  def on_perturb_clicked(self, button, axis=None):
    # are we perturbing x or y use axis or infer from button description
    if axis is None:
      if button.description == 'Perturb w':
        axis = 'w'
      elif button.description == 'Perturb b':
        axis = 'b'
      else:
        raise ValueError("Invalid button description, expected 'Perturb w' or 'Perturb b'")
    if axis == 'w':
      self.test_w = self.current_w + self.step_size
      self.test_b = self.current_b
    elif axis == 'b':
      self.test_b = self.current_b + self.step_size
      self.test_w = self.current_w
    else:
      raise ValueError("Invalid axis, expected 'w' or 'b'")
    # calculate test value
    self.test_r, _, _ = self.eval_function(self.test_w, self.test_b)
    self.test_r = self.test_r[0,0]
    self.total_function_evaluations += 1
    if axis == 'w':
      self.test_w_point.set_data([self.test_w], [self.test_r])
      self.measure_w.disabled = False
      self.perturb_w.disabled = True
      self.perturb_b.disabled = True
    elif axis == 'b':
      self.test_b_point.set_data([self.test_r], [self.test_b])
      self.measure_b.disabled = False
      self.perturb_b.disabled = True
      self.perturb_w.disabled = True
    self.fig.canvas.draw_idle()
    with self.narration_display:
      clear_output(wait=True)
      print(f'Test point: w = {self.test_w:.3f}, b = {self.test_b:.3f}, reward = {self.test_r:.3f}.')
      print(f"Click 'Measure' to propose a new {axis} value based on this test point.")

  def on_measure_clicked(self, button, axis=None):
    if axis is None:
      if button.description == 'Measure w':
        axis = 'w'
      elif button.description == 'Measure b':
        axis = 'b'
      else:
        raise ValueError("Invalid button description, expected 'Measure w' or 'Measure b'")
    if axis == 'w':
      self.update_measure_w()
      with self.narration_display:
        clear_output(wait=True)
        print(f'Based on measurement of w slope, proposed w step from {self.current_w:.3f} to {self.proposed_w:.3f}')
        print("Click 'Step w', to make the step.")
      self.step_w.disabled = False
      self.measure_w.disabled = True
    else: # axis == 'y':
      self.update_measure_b()
      with self.narration_display:
        clear_output(wait=True)
        print(f'Based on measurement of b slope, proposed b step from {self.current_b:.3f} to {self.proposed_b:.3f}')
        print("Click 'Step b', to make the step.")
        self.step_b.disabled = False
        self.measure_b.disabled = True

  def on_step_clicked(self, button):
    self.num_steps += 1
    if self.proposed_w is None and self.proposed_b is None:
      raise ValueError("At least one of proposed w and b values must be set before stepping.")
    if self.proposed_w is None:
      self.proposed_w = self.current_w
    if self.proposed_b is None:
      self.proposed_b = self.current_b
    new_r, _, _ = self.eval_function(self.proposed_w, self.proposed_b)
    new_r = new_r[0,0]
    self.total_function_evaluations += 1
    with self.narration_display:
      clear_output(wait=True)
      print(f'Stepped to w = {self.proposed_w:.3f}, b = {self.proposed_b:.3f}, reward = {new_r:.3f}.')
      print(f"Click 'Propose' or 'Do It For Me' to try a new value.")
      print(f'Total function evaluations so far: {self.total_function_evaluations}')
      print(f'Total number of steps taken so far: {self.num_steps}')
    self.w_history.append(self.current_w)
    self.b_history.append(self.current_b)
    self.r_history.append(self.current_r)
    self.points_history.set_data(self.w_history, self.b_history)
    self.current_w = self.proposed_w
    self.current_b = self.proposed_b
    self.current_r = new_r
    self.point_current.set_data([self.current_w], [self.current_b])
    self.vline_current.set_data([self.current_w, self.current_w], self.b_bounds)
    self.hline_current.set_data(self.w_bounds, [self.current_b, self.current_b])
    self.test_w, self.test_b, self.test_r = None, None, None
    self.test_w_point.set_data([], [])
    self.test_b_point.set_data([], [])
    self.proposed_w, self.proposed_b, self.proposed_r = None, None, None
    self.w_proposed_line.set_data([], [])
    self.b_proposed_line.set_data([], [])
    self.br_proposed_point.set_data([], [])
    self.wr_proposed_point.set_data([], [])
    self.point_proposed.set_data([], [])
    self.w_slope_line.set_data([], [])
    self.b_slope_line.set_data([], [])
    self.update_r_of_w()
    self.update_r_of_b()
    self.fig.canvas.draw_idle()
    self.step_w.disabled = True
    self.step_b.disabled = True
    self.perturb_w.disabled = False
    self.perturb_b.disabled = False
    self.full_step.disabled = False
    self.take_10_steps.disabled = False

  def on_full_step_clicked(self, button):
    self.on_perturb_clicked(button, axis='w')
    self.on_measure_clicked(button, axis='w')
    self.on_step_clicked(button)
    self.on_perturb_clicked(button, axis='b')
    self.on_measure_clicked(button, axis='b')
    self.on_step_clicked(button)

  def on_take_10_steps_clicked(self, button):
    for _ in range(10):
      self.on_full_step_clicked(button)

  def on_reset_clicked(self, button):
    self.proposals_evaluated = 0
    with self.narration_display:
      clear_output(wait=True)
      print(f'Reset. Clearing history')
      print(f'Randomizing initial w and b values.')
    self.current_w = self.rng.uniform(self.w_bounds[0], self.w_bounds[1])
    self.current_b = self.rng.uniform(self.b_bounds[0], self.b_bounds[1])
    current_r, _, _ = self.eval_function(self.current_w, self.current_b)
    self.current_r = current_r[0,0]
    self.point_current.set_data([self.current_w], [self.current_b])
    self.vline_current.set_data([self.current_w, self.current_w], self.b_bounds)
    self.hline_current.set_data(self.w_bounds, [self.current_b, self.current_b])
    self.test_w, self.test_b, self.test_r = None, None, None
    self.test_w_point.set_data([], [])
    self.test_b_point.set_data([], [])
    self.proposed_w, self.proposed_b, self.proposed_r = None, None, None
    self.w_proposed_line.set_data([], [])
    self.b_proposed_line.set_data([], [])
    self.br_proposed_point.set_data([], [])
    self.wr_proposed_point.set_data([], [])
    self.point_proposed.set_data([], [])
    self.w_slope_line.set_data([], [])
    self.b_slope_line.set_data([], [])
    self.w_history, self.b_history, self.r_history = [], [], []
    self.points_history.set_data([], [])
    self.update_r_of_w()
    self.update_r_of_b()
    self.fig.canvas.draw_idle()

  def on_show_curve_toggled(self, change):
    alpha = 1 if self.show_curve.value else 0
    self.line_polynomial.set_alpha(alpha)
    self.fig.canvas.draw_idle()

  def on_show_history_toggled(self, change):
    alpha = 0.5 if self.show_history.value else 0
    self.points_history.set_alpha(alpha)
    self.fig.canvas.draw_idle()


iwbss = InteractiveWBSlopeStepper()
display(iwbss.fig.canvas)
clear_output()
display(iwbss.ui)

So this isn't really much better than propose and reject. Like propose and reject there is rapid progress early on, but once the process reaches the flatish narrow ridge of the reward-parameter landscape, improvement slows down. Now, notably there is not much improvement to be had here. The optimal threshold achieves a score of 557, and this process finds a parameter configuration with reward very close to this optimal value relatively quickly. Scale really matters. How important is the difference between 557 and 556.709 in terms of the fitness of the organism. If this is neglibable, getting close is good enough, if this is the difference between life and death then getting close is not enough from an evolutionary perspective. Without further context we can't say much about whether this learning algorithm is quick enough or not. This is the kind of context we will slowly develop.

In [ ]:
# @markdown Submit your feedback
content_review(f"{feedback_prefix}_M5")

# Quiz

In [ ]:
# @markdown **Run this cell** to take the quiz
comprehension_quiz = [
  {
    "question": "How does the complexity of a neural circuit (number of parameters) impact the number of learning iterations and hence time to learn using a 'measure and update' learning process?",
    "type": "multiple_choice",
    "answers": [
      {
        "answer": "It does not affect the number of iterations required; the process scales well regardless of circuit complexity.",
        "correct": False,
        "feedback": "Actually, the the number of iterations required grows with the complexity of the circuit due to more parameters requiring optimization."
      },
      {
        "answer": "The process requires few learning iterations, by leveraging algorithmic economies of scale",
        "correct": False,
        "feedback": "Contrary to this, an increase in parameters leads the more learning iterations being required."
      },
      {
        "answer": "The process becomes requires more learning episodes for larger, more complex circuits.",
        "correct": True,
        "feedback": "Correct! More parameters mean more complexity and thus more learning episodes are needed."
      },
      {
        "answer": "The number of learning iterations needed is solely dependent on the type of learning task, not the circuit complexity.",
        "correct": False,
        "feedback": "Circuit complexity, particularly the number of parameters, plays a significant role in the number of learning iterations needed."
      }
    ]
  },
  {
    "question": "Why was the introduction of a 'hidden layer' beneficial in our complex discrimination tasks?",
    "type": "multiple_choice",
    "answers": [
      {
        "answer": "It allowed the model to perform tasks more quickly but with reduced accuracy.",
        "correct": False,
        "feedback": "The hidden layer's primary benefit is not speed at the cost of accuracy, but rather an enhancement in handling complex patterns."
      },
      {
        "answer": "It introduces feature interactions and non-linearities, enabling the model to capture complex patterns.",
        "correct": True,
        "feedback": "Exactly! Hidden layers allow for complex interactions and non-linear processing of features, which can be crucial for generating behaviour contingent on rich sensory input."
      },
      {
        "answer": "It reduces the number of parameters needed, simplifying the model.",
        "correct": False,
        "feedback": "Adding a hidden layer typically increases the number of parameters, adding complexity to the model."
      },
      {
        "answer": "It primarily improves the model's visualization, making it easier to interpret.",
        "correct": False,
        "feedback": "While interpretability is important, this is not an advantage of adding a hidden layer."
      }
    ]
  },
  {
    "question": "In the context of machine learning, what is the impact of using a 'mini-batch' approach?",
    "type": "multiple_choice",
    "answers": [
      {
        "answer": "It guarantees a 10x speedup in learning algorithms.",
        "correct": False,
        "feedback": "While mini-batches can speed up learning, the magnitude of this speed-up will depend on choice of mini-batch size and the interaction of this size with underlying algorithmic efficiencies of scale at the hardware implementation level."
      },
      {
        "answer": "It reduces the time for evaluations by using a smaller, representative data sample.",
        "correct": True,
        "feedback": "Correct! A mini-batch approach uses a smaller subset of data for quicker evaluations, though it introduces some noise to the estimates."
      },
      {
        "answer": "It decreases the accuracy of parameter evaluation",
        "correct": True,
        "feedback": "Correct! Mini-batches do introduce some noise to parameter evaluation, but is used thoughtfully this usually isn't an issue."
      },
      {
        "answer": "It eliminates the need for parameter updates in the learning process.",
        "correct": False,
        "feedback": "Mini-batches still require parameter updates; they just alter the way data is processed during learning."
      }
    ]
  },
  {
    "question": "What challenge arises when comparing different machine learning algorithms?",
    "type": "multiple_choice",
    "answers": [
      {
        "answer": "Algorithms cannot be compared due to their differing objectives.",
        "correct": False,
        "feedback": "While objectives can vary, this doesn't make comparison impossible; it's more about how different parameters and conditions affect performance."
      },
      {
        "answer": "All algorithms perform similarly when given the same data and task.",
        "correct": False,
        "feedback": "Performance can vary significantly between algorithms depending on their design and the specificities of the task and data."
      },
      {
        "answer": "Performance is highly dependent on meta-parameter choices, making direct comparisons challenging.",
        "correct": True,
        "feedback": "Variations in learning rate, perturbation scale, mini-batch size, and other meta-parameters can significantly impact algorithm performance, complicating direct comparisons."
      },
      {
        "answer": "Algorithms' performance cannot be measured or quantified.",
        "correct": False,
        "feedback": "Performance can indeed be measured and quantified, but the challenge lies in accounting for differences in meta-parameters and conditions."
      }
    ]
  }
]

display_quiz(comprehension_quiz)